In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join

In [127]:
#WordNet 2.1
import nltk
from nltk.corpus import WordNetCorpusReader
wn2 = WordNetCorpusReader("/Users/gary/Documents/perl/package/WordNet-2.1/dict", nltk.data.find("/Users/gary/Documents/perl/package/WordNet-2.1/dict"))

wn17 = WordNetCorpusReader("/Users/gary/Documents/NLP/WordNetVersions/wordnet-1.7/dict", nltk.data.find("/Users/gary/Documents/NLP/WordNetVersions/wordnet-1.6/dict"))

print(wn17.get_version())

1.7


In [3]:
799+820

1619

In [74]:
#extract sense cluster from the first file
path='/Users/gary/Documents/2020Fall/IntroNLP/project/DataSet/Senseval-2/'

senseval_file1=path+'corpora/english-lex-sample/train/eng-lex-sample.training.key'

sf1_senses=[]
sf1=open(senseval_file1,'r')
lines = sf1.readlines()
#extract sense to a list
for line in lines:
    line=line.strip('\n')
    #print(line.strip('\n'))
    line=line.split(' ')[2:]
    #print(line)
    sf1_senses.append(line) #list of list
    
#remove duplicated sense keys
senses_sf1=[]
for s1 in sf1_senses:
    if s1 not in senses_sf1:
        senses_sf1.append(s1)
#assign a group id to each sense key
df_sensekeys_sf1=pd.DataFrame(columns=['SenseKeys','Group'])
groupid=1
sense_dict={}

for ss in senses_sf1:
    flag_in=False
    #print(ss)
    for s in ss:
        if s in sense_dict.keys():
            flag_in=True
            gpid = sense_dict[s]
            break
    for s in ss:
        if len(s)>5:
            #print(s,groupid)
            if flag_in:
                sense_dict[s]=gpid
            else:
                sense_dict[s]=groupid
            
                
            #df_sensekeys_sf1=df_sensekeys_sf1.append({'SenseKeys':s,'Group':groupid},ignore_index=True)
    groupid+=1

df_sensekeys_sf1=pd.DataFrame(columns=['SenseKeys','Group'])
for key, value in sense_dict.items():
    df_sensekeys_sf1=df_sensekeys_sf1.append({'SenseKeys':key,'Group':value},ignore_index=True)

df_sensekeys_sf1.to_csv('df_senseval_sf1.csv',index=False) 


1

In [2]:
df_sensekeys_sf1=pd.read_csv('df_senseval_sf1.csv')
df_sensekeys_sf1

,SenseKeys,Group
0,art%1:06:00::,1
1,art_gallery%1:06:00::,2
2,art%1:04:00::,1
3,arts%1:09:00::,1
4,art_form%1:09:00::,5
...,...,...
794,work_on%2:36:00::,1048
795,work%2:41:04::,1049
796,work_on%2:41:00::,1050
797,work%2:35:14::,1053


In [86]:
headword='fine_art'
syn=''
sensekey='fine_art%1:06:00::'
for syt in wn17.synsets(headword):
        if len(syn)>2:
            break
        #if syt.pos() in ['a','s','n']:
        for lemma in syt.lemmas():
            if lemma.key()==sensekey:
                print(syt)

Synset('art.n.01')


In [80]:
#add synset by sensekey
#based on ReadMe file, the sensekys are from nouns and adjectives.
def add_synset(df_data):
    for i in range(len(df_data)):
        sensekey=df_data.loc[df_data.index[i],'SenseKeys']
        headword = sensekey.split('%')[0]
        syn=''
        for syt in wn17.synsets(headword):
            if len(syn)>2:
                break
            #if syt.pos() in ['a','s','n']:
            for lemma in syt.lemmas():
                if lemma.key()==sensekey:
                    syn=syt.name()
                    pos=syn.split('.')[1]
                    syn=syn.replace('.','#')
                    df_data.loc[df_data.index[i],'Pos']=pos
                    df_data.loc[df_data.index[i],'Synset']=syn
                    break
    return df_data

df_sensekeys_sf1 = add_synset(df_sensekeys_sf1)

df_sensekeys_sf1

,SenseKeys,Group,Pos,Synset
0,art%1:06:00::,1,n,art#n#01
1,art_gallery%1:06:00::,2,n,gallery#n#03
2,art%1:04:00::,1,n,art#n#02
3,arts%1:09:00::,1,n,humanistic_discipline#n#01
4,art_form%1:09:00::,5,n,art_form#n#01
...,...,...,...,...
794,work_on%2:36:00::,1048,v,work#v#05
795,work%2:41:04::,1049,v,work#v#08
796,work_on%2:41:00::,1050,v,work_at#v#01
797,work%2:35:14::,1053,v,work#v#21


In [87]:
#drop NaN synset
df_sensekeys_sf1 = df_sensekeys_sf1[df_sensekeys_sf1['Synset'].notna()]

df_sf1_duplicate=df_sensekeys_sf1.groupby(by=['Synset'],as_index=False, sort=False)['Pos'].count()
synsf1_dp_list=df_sf1_duplicate[df_sf1_duplicate['Pos']>1]['Synset'].to_list()

#make duplicated synset have the same group
for i in range(len(synsf1_dp_list)):
    gpsf1_dp_list=df_sensekeys_sf1[df_sensekeys_sf1['Synset']==synsf1_dp_list[i]]['Group'].to_list()
    for p in gpsf1_dp_list[1:]:
        df_sensekeys_sf1['Group'].replace(p,gpsf1_dp_list[0],inplace=True)

#drop duplicated synset
df_sensekeys_sf1= df_sensekeys_sf1.drop_duplicates(subset=['Synset'], keep='first')

df_sensekeys_sf1['WordPos']=df_sensekeys_sf1['Synset'].str.split('#').str[0]+'-'+df_sensekeys_sf1['Pos']

df_sensekeys_sf1.to_csv('Senseval2_sf1_raw.csv',index=False)



In [89]:
#generate sense pairs
df_sensekeys_sf1=pd.read_csv('Senseval2_sf1_raw.csv')
df_sensevasf1_pair=pd.DataFrame(columns=['Pos','Sense1','Sense2','Merge'])
df_sensevasf1_pair=generate_sensepair(df_sensekeys_sf1,df_sensevasf1_pair)
df_sensevasf1_pair.to_csv('Senseval2_sf1_sense_pair.csv',index=False)
df_sensevasf1_pair


processing 100/473
processing 200/473
processing 300/473
processing 400/473


,Pos,Sense1,Sense2,Merge
0,v,wander#v#01,wander#v#03,not-merged
1,v,observe#v#08,observe#v#09,not-merged
2,v,draw#v#20,draw#v#08,not-merged
3,v,draw#v#20,draw#v#11,not-merged
4,v,draw#v#20,draw#v#06,not-merged
...,...,...,...,...
908,n,workday#n#02,workday#n#01,not-merged
909,v,draw_up#v#02,draw_up#v#03,not-merged
910,v,draw_up#v#02,draw_up#v#05,not-merged
911,v,draw_up#v#03,draw_up#v#05,not-merged


In [90]:
df_sensevasf1_pair['sens12']=df_sensevasf1_pair['Sense1']+'-'+df_sensevasf1_pair['Sense2']
len(set(df_sensevasf1_pair['sens12'].to_list()))

913

In [92]:
#analysis of paris
df_sensevasf1_pair=pd.read_csv('Senseval2_sf1_sense_pair.csv')
total_a=len(df_sensevasf1_pair[df_sensevasf1_pair['Pos']=='a'])
total_n=len(df_sensevasf1_pair[df_sensevasf1_pair['Pos']=='n'])
total_v=len(df_sensevasf1_pair[df_sensevasf1_pair['Pos']=='v'])

notmerge_a=len(df_sensevasf1_pair[(df_sensevasf1_pair['Pos']=='a')&(df_sensevasf1_pair['Merge']=='not-merged')])
notmerge_n=len(df_sensevasf1_pair[(df_sensevasf1_pair['Pos']=='n')&(df_sensevasf1_pair['Merge']=='not-merged')])
notmerge_v=len(df_sensevasf1_pair[(df_sensevasf1_pair['Pos']=='v')&(df_sensevasf1_pair['Merge']=='not-merged')])

merge_a=len(df_sensevasf1_pair[(df_sensevasf1_pair['Pos']=='a')&(df_sensevasf1_pair['Merge']=='merged')])
merge_n=len(df_sensevasf1_pair[(df_sensevasf1_pair['Pos']=='n')&(df_sensevasf1_pair['Merge']=='merged')])
merge_v=len(df_sensevasf1_pair[(df_sensevasf1_pair['Pos']=='v')&(df_sensevasf1_pair['Merge']=='merged')])

rate_a=merge_a/total_a
rate_n=merge_n/total_n
rate_v=merge_v/total_v
print('a',total_a, notmerge_a, merge_a, rate_a)
print('n',total_n, notmerge_n, merge_n, rate_n)
print('v',total_v, notmerge_v, merge_v, rate_v)


a 16 10 6 0.375
n 127 104 23 0.18110236220472442
v 739 724 15 0.02029769959404601


In [93]:
#map senseva wn1.7 to wn2.1 
df_pairs=pd.read_csv('Senseval2_sf1_sense_pair.csv')

df_wn17wn21_sf1=pd.DataFrame(columns=['Pos','sense1_key17','sense1_wn17','sense2_wn17','sense2_key17','sense1_wn21','sense2_wn21','offset1_wn21','offset2_wn21','sense1_wn17_def','sense1_wn21_def','sense2_wn17_def','sense2_wn21_def','Merge'])

#df_pairs_n=df_pairs[df_pairs['Pos']=='n']
#total_pair=len(df_pairs_n)
total_pair=len(df_pairs)
df_senseval2=pd.read_csv('Senseval2_sf1_raw.csv')
#df_senseval2[df_senseval2['Synset']=='pull#v#12']['SenseKeys'].values[0]

for i in range(total_pair):
    if (i%1000 ==0):
        print('processing {}/{}'.format(i,total_pair))
    sense1_wn17 = df_pairs.loc[df_pairs.index[i],'Sense1']
    headword=sense1_wn17.split('#')[0]
    sense1_wn17 = sense1_wn17.replace('#','.')
    sense2_wn17 = df_pairs.loc[df_pairs.index[i],'Sense2']
    merge= df_pairs.loc[df_pairs.index[i],'Merge'] 
    sense2_wn17 = sense2_wn17.replace('#','.')
    pos = df_pairs.loc[df_pairs.index[i],'Pos']
    
    lemmas=''
    sense1_wn2=''
    sense1_wn2_def=''
    offset1_wn2=''
    for lemma1 in wn17.synset(sense1_wn17).lemmas():
        if len(sense1_wn2)>2:
            break
        sense1_wn17_def = wn17.synset(sense1_wn17).definition()
        #get sense key for wn21 sense1
        wn17_sense1_key=lemma1.key()
        #enumerate all wn synsests for headword#pos,find the same sense key
        for synset1 in wn2.synsets(headword, pos):
            lemmas = synset1.lemmas()
            if len(sense1_wn2)>2:
                break
            for l1 in lemmas:
                #if sense key equal, keep the wn16 synset name
                if l1.key() == wn17_sense1_key:
                    sense1_wn2=synset1.name()
                    sense1_wn2_def= synset1.definition()
                    offset1_wn2 = str(synset1.offset()).zfill(8) +'-'+ synset1.pos()
                    break

    lemmas=''
    sense2_wn2=''
    offset2_wn2=''
    
    for lemma2 in wn17.synset(sense2_wn17).lemmas():
        if len(sense2_wn2)>2:
            break
        sense2_wn17_def = wn17.synset(sense2_wn17).definition()
        wn17_sense2_key=lemma2.key()
        for synset2 in wn2.synsets(headword, pos):
            lemmas = synset2.lemmas()
            if len(sense2_wn2)>2:
                break
            for l2 in lemmas:
                if l2.key() == wn17_sense2_key:
                    sense2_wn2=synset2.name()
                    sense2_wn2_def= synset2.definition()
                    offset2_wn2 = str(synset2.offset()).zfill(8) +'-'+ synset2.pos()
                    break

    df_wn17wn21_sf1=df_wn17wn21_sf1.append({'Pos':pos,'sense1_key17':wn17_sense1_key,'sense1_wn17':sense1_wn17,'sense2_wn17':sense2_wn17,'sense2_key17':wn17_sense2_key,'sense1_wn21':sense1_wn2,'sense2_wn21':sense2_wn2,
                                    'offset1_wn21':offset1_wn2,'offset2_wn21':offset2_wn2,'sense1_wn21_def':sense1_wn2_def,'sense2_wn21_def':sense2_wn2_def,'Merge':merge},
                               ignore_index=True)
   
    
df_wn17wn21_sf1.to_csv('Senseval_WN17WN21full_sf1.csv',index=False)

df_wn17wn21_sf1



processing 0/913


,Pos,sense1_key17,sense1_wn17,sense2_wn17,sense2_key17,sense1_wn21,sense2_wn21,offset1_wn21,offset2_wn21,sense1_wn17_def,sense1_wn21_def,sense2_wn17_def,sense2_wn21_def,Merge
0,v,wander%2:38:00::,wander.v.01,wander.v.03,wander%2:38:02::,roll.v.12,wander.v.03,01863577-v,02083938-v,NaN,move about aimlessly or without any destinatio...,NaN,go via an indirect route or at no set pace,not-merged
1,v,observe%2:31:00::,observe.v.08,observe.v.09,observe%2:41:04::,observe.v.08,observe.v.09,00724225-v,02554084-v,NaN,observe correctly or closely,NaN,conform one's action or practice to,not-merged
2,v,draw%2:36:02::,draw.v.20,draw.v.08,draw%2:38:00::,draw.v.19,draw.v.07,01628770-v,01836719-v,NaN,engage in drawing,NaN,take liquid out of a container or well,not-merged
3,v,draw%2:36:02::,draw.v.20,draw.v.11,draw%2:37:01::,draw.v.19,draw.v.10,01628770-v,01743542-v,NaN,engage in drawing,NaN,"elicit responses, such as objections, criticis...",not-merged
4,v,draw%2:36:02::,draw.v.20,draw.v.06,draw%2:38:01::,draw.v.19,draw.v.05,01628770-v,01976850-v,NaN,engage in drawing,NaN,"bring, take, or pull out of a container or fro...",not-merged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,n,workday%1:28:00::,workday.n.02,workday.n.01,workday%1:28:01::,workday.n.02,workday.n.01,14937834-n,14938034-n,NaN,the amount of time that a worker must work for...,NaN,a day on which work is done,not-merged
909,v,draw_up%2:38:02::,draw_up.v.02,draw_up.v.03,draw_up%2:38:01::,draw_up.v.02,draw_up.v.03,01964369-v,01845990-v,NaN,straighten oneself,NaN,cause (a vehicle) to stop,not-merged
910,v,draw_up%2:38:02::,draw_up.v.02,draw_up.v.05,draw_up%2:38:00::,draw_up.v.02,draw_up.v.05,01964369-v,01845738-v,NaN,straighten oneself,NaN,come to a halt after driving somewhere,not-merged
911,v,draw_up%2:38:01::,draw_up.v.03,draw_up.v.05,draw_up%2:38:00::,draw_up.v.03,draw_up.v.05,01845990-v,01845738-v,NaN,cause (a vehicle) to stop,NaN,come to a halt after driving somewhere,not-merged


In [94]:
#remove sense pair in OntoNotes
features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
df_onetonote_features= pd.read_csv(features_path+'feature_space.csv')
df_otf=df_onetonote_features[['Sense1','Sense2']]
df_otf['se1se2']=df_otf['Sense1']+'-'+df_otf['Sense2']

otf_set=set(df_otf['se1se2'].to_list())

df_senseva_WN21=pd.read_csv('Senseval_WN17WN21full_sf1.csv')
df_senseva_WN21['sense1_wn21']=df_senseva_WN21['sense1_wn21'].str.replace('.','#')
df_senseva_WN21['sense2_wn21']=df_senseva_WN21['sense2_wn21'].str.replace('.','#')

df_senseva_WN21['selse2']=df_senseva_WN21['sense1_wn21']+'-'+df_senseva_WN21['sense2_wn21']

senseva_set=set(df_senseva_WN21['selse2'].to_list())

#delete overlap from senseva set


senseva_set - otf_set

df_senseva_WN21=df_senseva_WN21[df_senseva_WN21['selse2'].isin(list(senseva_set - otf_set))]


df_senseva_WN21=df_senseva_WN21[df_senseva_WN21['sense1_wn21'].notna()]
df_senseva_WN21=df_senseva_WN21[df_senseva_WN21['sense2_wn21'].notna()]
df_senseva_WN21.to_csv('Senseval2_sense_pair_W21_nooverlap_sf1.csv',index=False)

df_senseva_WN21

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Pos,sense1_key17,sense1_wn17,sense2_wn17,sense2_key17,sense1_wn21,sense2_wn21,offset1_wn21,offset2_wn21,sense1_wn17_def,sense1_wn21_def,sense2_wn17_def,sense2_wn21_def,Merge,selse2
0,v,wander%2:38:00::,wander.v.01,wander.v.03,wander%2:38:02::,roll#v#12,wander#v#03,01863577-v,02083938-v,NaN,move about aimlessly or without any destinatio...,NaN,go via an indirect route or at no set pace,not-merged,roll#v#12-wander#v#03
1,v,observe%2:31:00::,observe.v.08,observe.v.09,observe%2:41:04::,observe#v#08,observe#v#09,00724225-v,02554084-v,NaN,observe correctly or closely,NaN,conform one's action or practice to,not-merged,observe#v#08-observe#v#09
2,v,draw%2:36:02::,draw.v.20,draw.v.08,draw%2:38:00::,draw#v#19,draw#v#07,01628770-v,01836719-v,NaN,engage in drawing,NaN,take liquid out of a container or well,not-merged,draw#v#19-draw#v#07
3,v,draw%2:36:02::,draw.v.20,draw.v.11,draw%2:37:01::,draw#v#19,draw#v#10,01628770-v,01743542-v,NaN,engage in drawing,NaN,"elicit responses, such as objections, criticis...",not-merged,draw#v#19-draw#v#10
4,v,draw%2:36:02::,draw.v.20,draw.v.06,draw%2:38:01::,draw#v#19,draw#v#05,01628770-v,01976850-v,NaN,engage in drawing,NaN,"bring, take, or pull out of a container or fro...",not-merged,draw#v#19-draw#v#05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,n,workday%1:28:00::,workday.n.02,workday.n.01,workday%1:28:01::,workday#n#02,workday#n#01,14937834-n,14938034-n,NaN,the amount of time that a worker must work for...,NaN,a day on which work is done,not-merged,workday#n#02-workday#n#01
909,v,draw_up%2:38:02::,draw_up.v.02,draw_up.v.03,draw_up%2:38:01::,draw_up#v#02,draw_up#v#03,01964369-v,01845990-v,NaN,straighten oneself,NaN,cause (a vehicle) to stop,not-merged,draw_up#v#02-draw_up#v#03
910,v,draw_up%2:38:02::,draw_up.v.02,draw_up.v.05,draw_up%2:38:00::,draw_up#v#02,draw_up#v#05,01964369-v,01845738-v,NaN,straighten oneself,NaN,come to a halt after driving somewhere,not-merged,draw_up#v#02-draw_up#v#05
911,v,draw_up%2:38:01::,draw_up.v.03,draw_up.v.05,draw_up%2:38:00::,draw_up#v#03,draw_up#v#05,01845990-v,01845738-v,NaN,cause (a vehicle) to stop,NaN,come to a halt after driving somewhere,not-merged,draw_up#v#03-draw_up#v#05


In [96]:
#get features using perl language
df_senseva_WN21_sf1=pd.read_csv('Senseval2_sense_pair_W21_nooverlap_sf1.csv')
df_senseva_pair21_sf1=df_senseva_WN21_sf1[['Pos','sense1_wn21','sense2_wn21','Merge']]
df_senseva_pair21_sf1.columns=['Pos','Sense1','Sense2','Merge']
df_senseva_pair21_sf1.to_csv('Senseva2_sensepairs_wn21_sf1.csv')

In [97]:
'''
Build feature space:
1, twin: the number of shared synonyms between two synsets
2, antonym: whether two synsets share an antonym
3, pertainym: whether two synsets share an pertainym
4, deriv: whether two synsets share derivationally related forms
5, verbgrp: whether two verb synsets are linked in a VERBGROUP (indicating semantic similarity)
6, verbfrm: whether two verb synsets share a VERBFRAM (indicating syntactic similarity) 

'''
#wordnet 2.1
df_pairs=pd.read_csv('Senseva2_sensepairs_wn21_sf1.csv')
#df_pairs
for i in range(len(df_pairs)):
    pertainyms1=[]
    antonyms1=[]
    deriv1=[]
    lemmas1=[]
    verbgroup1=[]
    verbframe1=[]
    pertainyms2=[]
    antonyms2=[]
    deriv2=[]
    lemmas2=[]
    verbgroup2=[]
    verbframe2=[]
    pertainymsflag=''
    antonymsflag=''
    derivflag=''
    lemmasflag=''
    verbgroupflag=''
    verbframeflag=''
    hyper_min=''
    hyper_max=''

    sense1 = df_pairs.loc[df_pairs.index[i],'Sense1']
    sense1 = sense1.replace('#','.')
    sense2 = df_pairs.loc[df_pairs.index[i],'Sense2']
    sense2 = sense2.replace('#','.')
    pos = df_pairs.loc[df_pairs.index[i],'Pos']
    
    # sense1's twin, pertainyms, antonyms, derivationally_related_forms,
    # verb group, frame id, hyper_min, hyper_max
    try:
        
        #WN, WNMax feature
        #calculate hyper distance
        sense1_hyper = wn2.synset(sense1)
        sense1_hypers = lambda s: s.hypernyms()
        hyper1= list(sense1_hyper.closure(sense1_hypers))
        
        sense2_hyper = wn2.synset(sense2)
        sense2_hypers = lambda s: s.hypernyms()
        hyper2 = list(sense2_hyper.closure(sense2_hypers))
           
        #find the nearest hyper, and average the distance as the least distance
        for h1 in hyper1:
            if h1 in hyper2:
                hyper_min = (hyper1.index(h1)+hyper2.index(h1))/2
                break
        


        #find the farest hyper, and average the distance as the largest distance
        for h1 in reversed(hyper1):
            if h1 in hyper2:
                hyper_max = (hyper1.index(h1)+hyper2.index(h1))/2
                break
                
        
        for lemma in wn2.synset(sense1).lemmas():
            
            pertainyms1=pertainyms1+lemma.pertainyms()
            antonyms1=antonyms1+lemma.antonyms()
            deriv1=deriv1+lemma.derivationally_related_forms()
            lemmas1.append(lemma.name())
            
            if pos=='v':
                verbgroup1=verbgroup1+lemma.verb_groups()
                verbframe1=verbframe1+lemma.frame_ids()

                
        # sense2's pertainyms, antonyms, derivationally_related_forms
        for lemma in wn2.synset(sense2).lemmas():
            
            pertainyms2=pertainyms2+lemma.pertainyms()
            antonyms2=antonyms2+lemma.antonyms()
            deriv2=deriv2+lemma.derivationally_related_forms()
            lemmas2.append(lemma.name())
             
            #verb group, verb frame
            if pos=='v':
                verbgroup2=verbgroup2+lemma.verb_groups()
                verbframe2=verbframe2+lemma.frame_ids()
                
      
        if (len(set(pertainyms1)&set(pertainyms2))>0):
            pertainymsflag=1
        else:
            pertainymsflag=0

        if (len(set(antonyms1)&set(antonyms2))>0):
            antonymsflag=1
        else:
            antonymsflag=0 

        if (len(set(deriv1)&set(deriv2))>0):
            derivflag=1
        else:
            derivflag=0 
        
        #verb group, verb frame
        if pos=='v':
            if (len(set(verbgroup1)&set(verbgroup2))>0):
                verbgroupflag=1
            else:
                verbgroupflag=0
                
            if (len(set(verbframe1)&set(verbframe2))>0):
                verbframeflag=1
            else:
                verbframeflag=0
            

        
        lemmasflag=len(set(lemmas1)&set(lemmas2))
       
        
        df_pairs.loc[df_pairs.index[i],'pertainyms']=pertainymsflag
        df_pairs.loc[df_pairs.index[i],'antonyms']=antonymsflag
        df_pairs.loc[df_pairs.index[i],'deriv']=derivflag
        df_pairs.loc[df_pairs.index[i],'lemmas']=lemmasflag
        
        df_pairs.loc[df_pairs.index[i],'verbgroup']=verbgroupflag
        df_pairs.loc[df_pairs.index[i],'verbframe']=verbframeflag
         
        df_pairs.loc[df_pairs.index[i],'hyper_min']=hyper_min
        df_pairs.loc[df_pairs.index[i],'hyper_max']=hyper_max

        
    except:
        None
    
df_pairs.to_csv('Senseva2_SensesPairs_WNFeatures_sf1.csv')   

In [98]:
#create feature based on mapping between WN and OED, the mapping is based on WN21
#read mapping file
file_map_wnoed=open('/Users/gary/Documents/2020Fall/IntroNLP/project/FeatureSpace/sense_clusters-21.senses','r')

#read sysnset pairs
df_pairs=pd.read_csv('Senseva2_sensepairs_wn21_sf1.csv')


#df_pairs_n=df_pairs[df_pairs['Pos']=='n']
#total_pair=len(df_pairs_n)
total_pair=len(df_pairs)
for i in range(total_pair):
    if (i%1000 ==0):
        print('processing {}/{}'.format(i,total_pair))
    sense1_wn21 = df_pairs.loc[df_pairs.index[i],'Sense1']
    headword=sense1_wn21.split('#')[0]
    sense1_wn21 = sense1_wn21.replace('#','.')
    sense2_wn21 = df_pairs.loc[df_pairs.index[i],'Sense2']
    sense2_wn21 = sense2_wn21.replace('#','.')
    pos = df_pairs.loc[df_pairs.index[i],'Pos']
    
    sense1_keys=''
    sense2_keys=''
    try:
        for lemma1 in wn2.synset(sense1_wn21).lemmas():
            #get sense key for wn21 sense1
            sense1_keys=sense1_keys+lemma1.key()+';'
        for lemma2 in wn2.synset(sense2_wn21).lemmas():
            #get sense key for wn21 sense1
            sense2_keys=sense2_keys+lemma1.key()+';'
    except:
        None

    df_pairs.loc[df_pairs.index[i],'Sense1_keys']=sense1_keys
    df_pairs.loc[df_pairs.index[i],'Sense2_keys']=sense2_keys

#filter sense keys is null
df_pairs = df_pairs[(df_pairs['Sense1_keys'].str.len()>5)&(df_pairs['Sense1_keys'].str.len()>5)]

#get sense clusterings from the mapping between WN and OED
lines = file_map_wnoed.readlines()
count=0
wnoed_lst=[] # a list of sets.
for line in lines:
    count+=1
    line=line.strip('\n')
    if ' ' in line:
        wnoed_lst.append(set(line.split(' ')))

for i in range(len(df_pairs)):
    if i%1000 ==0:
        print('processing {}/{}'.format(i,total_pair))
    sense1_key=df_pairs.loc[df_pairs.index[i],'Sense1_keys']
    sk1_set=set(sense1_key.split(';'))
    
    sense2_key=df_pairs.loc[df_pairs.index[i],'Sense2_keys']
    sk2_set=set(sense2_key.split(';'))
    
    feature=0
    for  w in wnoed_lst:
        sk1_len=len(sk1_set & w)
        sk2_len=len(sk2_set & w)
        if (sk1_len>0 and sk2_len>0):
            feature=1
            break
    
    df_pairs.loc[df_pairs.index[i],'wn_oed_feature']=feature
    
df_pairs.to_csv('WN_OED_Map_Feature_Senseva2_SensesPairs_sf1.csv',index=False)



processing 0/874
processing 0/874


In [99]:
#concatnate all the features together.

features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
feature_files=['Senseva2_sensepairs_wn21_sf1.csv','Senseva2_pairs_wnsimilarity_sf1.csv','Senseva2_SensesPairs_WNFeatures_sf1.csv','WN_OED_Map_Feature_Senseva2_SensesPairs_sf1.csv']


df_sensepari=pd.read_csv(features_path+feature_files[0])
df_sensepari= df_sensepari[['Pos', 'Sense1', 'Sense2', 'Merge']]
df_sensepari=df_sensepari.drop_duplicates()

print('df_sensepari',len(df_sensepari))
df_wnpackage=pd.read_csv(features_path+feature_files[1])
df_wnpackage=df_wnpackage.drop_duplicates()
print('df_wnpackage',len(df_wnpackage))

df_features_tmp = df_sensepari.merge(df_wnpackage,left_on=['Sense1', 'Sense2','Merge'], 
                              right_on=['sense1', 'sense2','merge'], how='left')


df_features_tmp=df_features_tmp[['Pos', 'Sense1', 'Sense2', 'Merge', 
       'lch', 'hso', 'jcn', 'leskvalue', 'linvalue', 'resvalue', 'vecvalue',
       'wupvalue']]
df_features_tmp=df_features_tmp.drop_duplicates()
print('df_features_tmp',len(df_features_tmp))

df_wncorpus=pd.read_csv(features_path+feature_files[2])

print('df_wncorpus',len(df_wncorpus))

df_features_tmp1 = df_features_tmp.merge(df_wncorpus,left_on=['Pos', 'Sense1', 'Sense2', 'Merge'], 
                              right_on=['Pos', 'Sense1', 'Sense2', 'Merge'], how='left')

df_features_tmp1.columns
print('df_features_tmp1',len(df_features_tmp1))


df_features_tmp1=df_features_tmp1[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 
        'pertainyms', 'antonyms', 'deriv', 'lemmas',
       'verbgroup', 'verbframe', 'hyper_min', 'hyper_max']]

df_features_tmp1=df_features_tmp1.drop_duplicates() 

#wn-oed mapping feature
df_wnoedfea=pd.read_csv(features_path+feature_files[3])
df_wnoedfea
print('df_wnoedfea',len(df_wnoedfea))
df_features = df_features_tmp1.merge(df_wnoedfea[['Pos', 'Sense1', 'Sense2', 'Merge','wn_oed_feature']],left_on=['Pos', 'Sense1', 'Sense2','Merge'], 
                              right_on=['Pos', 'Sense1', 'Sense2', 'Merge'], how='left')

df_features=df_features[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 'pertainyms',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe', 'hyper_min',
       'hyper_max', 'wn_oed_feature']]

df_features=df_features.drop_duplicates()
df_features
print('df_features',len(df_features))
df_features.to_csv('senseval2_feature_space_sf1.csv',index=False)
df_features



df_sensepari 874
df_wnpackage 875
df_features_tmp 874
df_wncorpus 874
df_features_tmp1 874
df_wnoedfea 874
df_features 874


,Pos,Sense1,Sense2,Merge,lch,hso,jcn,leskvalue,linvalue,resvalue,...,wupvalue,pertainyms,antonyms,deriv,lemmas,verbgroup,verbframe,hyper_min,hyper_max,wn_oed_feature
0,v,roll#v#12,wander#v#03,not-merged,1.945910,4.0,0.000000,10.0,0.0,3.318246,...,0.571429,0.0,0.0,0.0,1.0,0.0,1.0,0.5,0.5,0.0
1,v,observe#v#08,observe#v#09,not-merged,2.233592,0.0,0.053396,9.0,0.0,0.000000,...,0.500000,0.0,0.0,0.0,2.0,0.0,1.0,NaN,NaN,0.0
2,v,draw#v#19,draw#v#07,not-merged,1.540445,0.0,0.047931,8.0,0.0,0.000000,...,0.285714,0.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN,1.0
3,v,draw#v#19,draw#v#10,not-merged,1.386294,0.0,0.045832,2.0,0.0,0.000000,...,0.250000,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,1.0
4,v,draw#v#19,draw#v#05,not-merged,1.540445,0.0,0.048413,5.0,0.0,0.000000,...,0.285714,0.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,n,workday#n#02,workday#n#01,not-merged,1.558145,0.0,0.000000,7.0,0.0,3.514798,...,0.666667,0.0,0.0,0.0,2.0,NaN,NaN,2.5,7.5,1.0
870,v,draw_up#v#02,draw_up#v#03,not-merged,1.386294,0.0,0.046550,4.0,0.0,0.000000,...,0.250000,0.0,0.0,0.0,2.0,0.0,0.0,NaN,NaN,0.0
871,v,draw_up#v#02,draw_up#v#05,not-merged,1.386294,0.0,0.051186,4.0,0.0,0.000000,...,0.250000,0.0,0.0,0.0,2.0,0.0,0.0,NaN,NaN,0.0
872,v,draw_up#v#03,draw_up#v#05,not-merged,1.722767,4.0,0.046550,23.0,0.0,0.000000,...,0.333333,0.0,0.0,0.0,2.0,0.0,0.0,NaN,NaN,0.0


In [74]:
#deal with file 3
path='/Users/gary/Documents/2020Fall/IntroNLP/project/DataSet/Senseval-2/'

senseval_file1=path+'corpora/english-lex-sample/train/eng-lex-sample.training.key'
senseval_file2=path+'Sval2.keys/Senseval2.key'
senseval_file3=path+'corpora/english-lex-sample/train/eng-lex-sample.training.senses'

df_senseval=pd.DataFrame(columns=['SenseKeys','Group'])


sf3=open(senseval_file3,'r')
lines = sf3.readlines()
group=0
flag=0 #control to skip loop
for i in range(len(lines)):
    if flag>0:
        flag=flag-1
        continue
    line=lines[i].replace('\n','').split(' ')
    print(line)
    if len(line)==1:
        #print(line[0],group)
        
        df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
        group+=1
    else:
        if line[1]=='2':
            print(line[0],group)
            print(line[2],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            df_senseval=df_senseval.append({'SenseKeys':line[2],'Group':group},ignore_index=True)
            group+=1
            
        if line[1]=='3':
            print(line[0],group)
            print(line[2],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            df_senseval=df_senseval.append({'SenseKeys':line[2],'Group':group},ignore_index=True)
            line=lines[i+1].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            group+=1
            flag+=1
        if line[1]=='4':
            print(line[0],group)
            print(line[2],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            df_senseval=df_senseval.append({'SenseKeys':line[2],'Group':group},ignore_index=True)

            line=lines[i+1].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+2].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
   
            group+=1
            flag+=2
        if line[1]=='5':
            print(line[0],group)
            print(line[2],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            df_senseval=df_senseval.append({'SenseKeys':line[2],'Group':group},ignore_index=True)


            line=lines[i+1].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+2].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+3].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            group+=1
            flag+=3
        if line[1]=='6':
            print(line[0],group)
            print(line[2],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            df_senseval=df_senseval.append({'SenseKeys':line[2],'Group':group},ignore_index=True)


            line=lines[i+1].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+2].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+3].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+4].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
 
            group+=1
            flag+=4
        if line[1]=='7':
            print(line[0],group)
            print(line[2],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            df_senseval=df_senseval.append({'SenseKeys':line[2],'Group':group},ignore_index=True)


            line=lines[i+1].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+2].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+3].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+4].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+5].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            group+=1
            flag+=5
        if line[1]=='8':
            print(line[0],group)
            print(line[2],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            df_senseval=df_senseval.append({'SenseKeys':line[2],'Group':group},ignore_index=True)


            line=lines[i+1].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+2].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)

            line=lines[i+3].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
 
            line=lines[i+4].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            line=lines[i+5].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            line=lines[i+6].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            group+=1
            flag+=6
        if line[1]=='9':
            print(line[0],group)
            print(line[2],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            df_senseval=df_senseval.append({'SenseKeys':line[2],'Group':group},ignore_index=True)


            line=lines[i+1].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            line=lines[i+2].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            line=lines[i+3].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            line=lines[i+4].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            line=lines[i+5].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            line=lines[i+6].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            line=lines[i+7].replace('\n','').split(' ')
            print(line[0],group)
            df_senseval=df_senseval.append({'SenseKeys':line[0],'Group':group},ignore_index=True)
            group+=1
            flag+=7
            
df_senseval.to_csv('df_senseval_an.csv',index=False)          

    



['attack%5:00:00:offensive:03']
['bum%5:00:00:inferior:02']
['county%3:01:00::']
['material%5:00:00:physical:00', '3', 'material%5:00:00:worldly:00']
material%5:00:00:physical:00 3
material%5:00:00:worldly:00 3
material%3:00:03:: 3
['material%3:00:04::', '2', 'material%3:00:01::']
material%3:00:04:: 4
material%3:00:01:: 4
['material%3:00:02::']
['post%5:00:00:succeeding(a):00']
['present%3:00:01::']
['present%3:00:02::']
['present%3:01:00::']
['stone%3:01:00::']
['stone%5:00:00:chromatic:00']
['air%1:15:00::', '4', 'air%1:27:00::']
air%1:15:00:: 12
air%1:27:00:: 12
air%1:19:00:: 12
air%1:27:01:: 12
['air%1:04:00::']
['air%1:10:02::']
['air%1:07:00::']
['air%1:10:01::']
['appeal%1:04:00::', '3', 'appeal%1:10:00::']
appeal%1:04:00:: 17
appeal%1:10:00:: 17
appeal%1:10:02:: 17
['appeal%1:07:00::']
['art%1:10:00::', '2', 'art%1:06:00::']
art%1:10:00:: 19
art%1:06:00:: 19
['art%1:09:00::', '2', 'art%1:04:00::']
art%1:09:00:: 20
art%1:04:00:: 20
['attack%1:10:00::', '4', 'attack%1:04:00::']
a

['drift%2:38:06::', '2', 'drift%2:38:02::']
drift%2:38:06:: 147
drift%2:38:02:: 147
['drift%2:38:01::', '2', 'drift%2:38:04::']
drift%2:38:01:: 148
drift%2:38:04:: 148
['drift%2:30:10::', '2', 'drift%2:42:03::']
drift%2:30:10:: 149
drift%2:42:03:: 149
['drift%2:38:05::', '2', 'drift%2:42:00::']
drift%2:38:05:: 150
drift%2:42:00:: 150
['drift%2:35:03::']
['drift%2:30:00::']
['drive%2:38:00::', '7', 'drive%2:38:01::']
drive%2:38:00:: 153
drive%2:38:01:: 153
drive%2:38:02:: 153
drive%2:42:00:: 153
drive%2:41:13:: 153
drive%2:38:09:: 153
drive%2:38:11:: 153
['drive%2:41:02::', '3', 'drive%2:35:00::']
drive%2:41:02:: 154
drive%2:35:00:: 154
drive%2:36:00:: 154
['drive%2:35:01::', '4', 'drive%2:35:11::']
drive%2:35:01:: 155
drive%2:35:11:: 155
drive%2:35:03:: 155
drive%2:35:02:: 155
['drive%2:33:01::', '3', 'drive%2:35:07::']
drive%2:33:01:: 156
drive%2:35:07:: 156
drive%2:33:00:: 156
['drive%2:41:00::']
['drive%2:32:00::']
['drive%2:35:06::']
['drive%2:34:07::']
['dyke%1:18:00::']
['dyke%1:

['see%2:33:00::']
['sense%1:09:02::', '2', 'sense%1:09:05::']
sense%1:09:02:: 307
sense%1:09:05:: 307
['sense%1:10:00::']
['sense%1:09:04::']
['sense%1:09:06::']
['serve%2:42:01::', '3', 'serve%2:42:03::']
serve%2:42:01:: 311
serve%2:42:03:: 311
serve%2:42:12:: 311
['serve%2:41:13::', '2', 'serve%2:33:00::']
serve%2:41:13:: 312
serve%2:33:00:: 312
['serve%2:42:02::', '3', 'serve%2:41:00::']
serve%2:42:02:: 313
serve%2:41:00:: 313
serve%2:35:01:: 313
['serve%2:34:00::', '3', 'serve%2:34:01::']
serve%2:34:00:: 314
serve%2:34:01:: 314
serve%2:35:00:: 314
['serve%2:41:01::', '2', 'serve%2:41:02::']
serve%2:41:01:: 315
serve%2:41:02:: 315
['serve%2:42:00::']
['serve%2:33:01::']
['shade%1:26:01::', '2', 'shade%1:26:00::']
shade%1:26:01:: 318
shade%1:26:00:: 318
['shade%1:23:00::', '2', 'shade%1:10:00::']
shade%1:23:00:: 319
shade%1:10:00:: 319
['shade%1:07:00::']
['shade%1:06:00::']
['shade%1:09:01::']
['shade%1:06:01::']
['shock%1:26:00::', '3', 'shock%1:12:01::']
shock%1:26:00:: 324
shock%

In [143]:
df_senseval

814

In [75]:
df_senseval

,SenseKeys,Group
0,attack%5:00:00:offensive:03,0
1,bum%5:00:00:inferior:02,1
2,county%3:01:00::,2
3,material%5:00:00:physical:00,3
4,material%5:00:00:worldly:00,3
...,...,...
815,work_out%2:31:07::,453
816,work_out%2:31:04::,453
817,work_out%2:42:00::,454
818,yew%1:20:00::,455


In [76]:
#get verb
verb_files=path+'corpora/english-lex-sample/train/verb-groups/'
output_path=path+'verbs/'
'''
filelist = [f for f in listdir(verb_files) if isfile(join(verb_files, f))] #get all files' name
 
for f in filelist:
    print(f)
    fv = open(verb_files+f,'r')
    output=open(output_path+f,'w')
    v_lines=fv.readlines()
    for line in v_lines:
        line=line.replace('\n','')
        if line.startswith('GROUP '):
            output.write(line+'\n')
        if line.startswith('Sense '):
            output.write(line+'\n')
    output.close()
''' 

filelist = [f for f in listdir(output_path) if isfile(join(output_path, f))] #get all files' name

df_senseval_v=pd.DataFrame(columns=['SenseKeys','Group','Synset'])
group=1001
for f in filelist:   
 
    flag=0
    fv=open(output_path+f,'r')
    v_lines=fv.readlines()
    for i in range(len(v_lines)):
        if flag>1:
            flag-=1
            continue
        line=v_lines[i].replace('\n','')
        if line.startswith('GROUP '):

            flag=1
            line=v_lines[i+flag].replace('\n','')
            if not(line.startswith('Sense ')):
                break
            else:
                try:
                    while line.startswith('Sense'):
                        print(f+'#v#'+line.split(' ')[1],group)
                        df_senseval_v=df_senseval_v.append({'SenseKeys':'','Group':group,'Synset':f+'#v#'+line.split(' ')[1]},ignore_index=True)
                        flag+=1
                        line=v_lines[i+flag].replace('\n','')
                    group+=1
                except:
                    print('error',f)
                    group+=1
df_senseval_v.to_csv('df_senseval_v.csv',index=False)

keep#v#1 1001
keep#v#10 1001
keep#v#3 1002
keep#v#17 1002
keep#v#4 1003
keep#v#14 1003
keep#v#21 1003
keep#v#5 1004
keep#v#13 1004
keep#v#7 1005
keep#v#15 1005
keep#v#16 1005
keep#v#9 1006
keep#v#11 1006
keep#v#12 1007
keep#v#22 1007
keep#v#18 1008
keep#v#19 1008
keep#v#20 1008
keep#v#2 1008
keep#v#6 1008
keep#v#8 1008
develop#v#1 1009
develop#v#2 1009
develop#v#3 1010
develop#v#4 1010
develop#v#5 1011
develop#v#9 1011
develop#v#10 1011
develop#v#14 1011
develop#v#20 1011
develop#v#6 1012
develop#v#7 1012
develop#v#8 1012
develop#v#11 1012
develop#v#12 1012
develop#v#13 1012
develop#v#7 1013
develop#v#19 1013
develop#v#17 1014
develop#v#18 1014
develop#v#15 1014
develop#v#16 1014
develop#v#21 1014
face#v#1 1015
face#v#2 1015
face#v#6 1015
face#v#3 1016
face#v#4 1016
face#v#5 1017
face#v#7 1017
face#v#8 1018
face#v#9 1018
match#v#1 1019
match#v#4 1019
match#v#6 1019
match#v#2 1020
match#v#9 1020
match#v#5 1021
match#v#8 1021
match#v#3 1021
match#v#7 1021
dress#v#1 1022
dress#v#2 1022
dr

In [77]:
df_senseval_v

,SenseKeys,Group,Synset
0,,1001,keep#v#1
1,,1001,keep#v#10
2,,1002,keep#v#3
3,,1002,keep#v#17
4,,1003,keep#v#4
...,...,...,...
462,,1127,wander#v#1
463,,1127,wander#v#3
464,,1127,wander#v#4
465,,1127,wander#v#2


In [263]:
#add synset by sensekey
#based on ReadMe file, the sensekys are from nouns and adjectives.
def add_synset():
    for i in range(len(df_senseval)):
        sensekey=df_senseval.loc[df_senseval.index[i],'SenseKeys']
        headword = sensekey.split('%')[0]
        syn=''
        for syt in wn17.synsets(headword):
            if len(syn)>2:
                break
            #if syt.pos() in ['a','s','n']:
            for lemma in syt.lemmas():
                if lemma.key()==sensekey:
                    syn=syt.name()
                    pos=syn.split('.')[1]
                    syn=syn.replace('.','#')
                    df_senseval.loc[df_senseval.index[i],'Pos']=pos
                    df_senseval.loc[df_senseval.index[i],'Synset']=syn
                    break

#add sensekey by synset
def add_sensekey():
    for i in range(len(df_senseval_v)):
        syn=df_senseval_v.loc[df_senseval_v.index[i],'Synset']
        headword = syn.split('#')[0]
        pos = syn.split('#')[1]
        syn=syn.replace('#','.')
        sens=''
        sensekey=[]
        for lemma1 in wn17.synset(syn).lemmas():
            sensekey.append(lemma1.key()) 

        for s in sensekey:
            if s.split('%')[0]==headword:
                sens=s
        if len(sens)==0:
            sens=sensekey[0]
        df_senseval_v.loc[df_senseval_v.index[i],'SenseKeys']=sens
        df_senseval_v.loc[df_senseval_v.index[i],'Pos']=pos

#df_senseval_v =pd.read_csv('df_senseval_v.csv')
df_senseval = pd.read_csv('df_senseval_an.csv') 

add_synset()
#add_sensekey()

#df_senseval2=pd.concat([df_senseval,df_senseval_v])
#add PoS to verb
#df_senseval2.loc[df_senseval2['SenseKeys'].isna(),'Pos']='v'
df_senseval2= df_senseval
#drop NaN synset
df_senseval2=df_senseval2[df_senseval2['Synset'].notna()]

#get duplicated synset
df_sva2_duplicate=df_senseval2.groupby(by=['Synset'],as_index=False, sort=False)['Pos'].count()
syns_dp_list=df_sva2_duplicate[df_sva2_duplicate['Pos']>1]['Synset'].to_list()
#make duplicated synset have the same group
for i in range(len(syns_dp_list)):
    gp_dp_list=df_senseval2[df_senseval2['Synset']==syns_dp_list[i]]['Group'].to_list()
    for p in gp_dp_list[1:]:
        df_senseval2['Group'].replace(p,gp_dp_list[0],inplace=True)
#drop duplicated synset
df_senseval2= df_senseval2.drop_duplicates(subset=['Synset'], keep='first')

df_senseval2['WordPos']=df_senseval2['Synset'].str.split('#').str[0]+'-'+df_senseval2['Pos']

df_senseval2.to_csv('Senseval2_raw.csv',index=False)

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [264]:
df_senseval2


,SenseKeys,Group,Pos,Synset,WordPos
0,attack%5:00:00:offensive:03,0,s,assault#s#01,assault-s
1,bum%5:00:00:inferior:02,1,s,bum#s#01,bum-s
2,county%3:01:00::,2,a,county#a#01,county-a
3,material%5:00:00:physical:00,3,s,material#s#04,material-s
4,material%5:00:00:worldly:00,3,s,material#s#01,material-s
...,...,...,...,...,...
814,work_out%2:31:06::,445,v,calculate#v#01,calculate-v
815,work_out%2:31:07::,445,v,work_out#v#05,work_out-v
817,work_out%2:42:00::,454,v,work_out#v#02,work_out-v
818,yew%1:20:00::,455,n,yew#n#02,yew-n


In [265]:
print(len(set(df_senseval2['Synset'].to_list()))) #1095
print(len(df_senseval2))
df_senseval2[df_senseval2['Synset'].isna()]


794
794


,SenseKeys,Group,Pos,Synset,WordPos


In [3]:
def generate_sensepair(df_data,df_pairs):
    """
    Purpos: generate sense pairs
    Auguments:
    df_data: input data
    df_pairs: output data
    """
    #
    wordpos_list=list(set(df_data['WordPos'].to_list()))
    count=1
    for w in wordpos_list:
        if count%100==0:
            print('processing {}/{}'.format(count,len(wordpos_list)))
        count+=1
        df_word = df_data[df_data['WordPos']==w]
        for i in range(len(df_word)):
            pos =df_word.loc[df_word.index[i],'Pos']
            gp_i=df_word.loc[df_word.index[i],'Group']
            sense1=df_word.loc[df_word.index[i],'Synset']

            for j in range(i+1,len(df_word)):
                gp_j=df_word.loc[df_word.index[j],'Group']
                sense2 = df_word.loc[df_word.index[j],'Synset']

                if gp_i==gp_j:

                    df_pairs=df_pairs.append({'Pos':pos,'Sense1':sense1,'Sense2':sense2,'Merge':'merged'},
                                                      ignore_index=True)
                else:

                    df_pairs=df_pairs.append({'Pos':pos,'Sense1':sense1,'Sense2':sense2,'Merge':'not-merged'},
                                                      ignore_index=True)
    return df_pairs

df_senseval2=pd.read_csv('Senseval2_raw.csv')
df_senseva_pair=pd.DataFrame(columns=['Pos','Sense1','Sense2','Merge'])
df_senseva_pair=generate_sensepair(df_senseval2,df_senseva_pair)
df_senseva_pair.to_csv('Senseval2_sense_pair.csv',index=False)
df_senseva_pair




processing 100/344
processing 200/344
processing 300/344


,Pos,Sense1,Sense2,Merge
0,v,wander#v#01,wander#v#03,merged
1,v,observe#v#06,observe#v#09,merged
2,v,observe#v#06,observe#v#08,not-merged
3,v,observe#v#09,observe#v#08,not-merged
4,v,wash_up#v#03,wash_up#v#01,merged
...,...,...,...,...
2639,v,draw_up#v#03,draw_up#v#01,not-merged
2640,v,draw_up#v#03,draw_up#v#02,not-merged
2641,v,draw_up#v#01,draw_up#v#02,not-merged
2642,v,discover#v#02,discover#v#04,merged


In [4]:
['sens12']=df_senseva_pair['Sense1']+'-'+df_senseva_pair['Sense2']
len(set(df_senseva_pair['sens12'].to_list()))


2644

In [5]:
#analysis of paris
df_senseva_pair=pd.read_csv('Senseval2_sense_pair.csv')
total_a=len(df_senseva_pair[df_senseva_pair['Pos']=='a'])
total_n=len(df_senseva_pair[df_senseva_pair['Pos']=='n'])
total_v=len(df_senseva_pair[df_senseva_pair['Pos']=='v'])

notmerge_a=len(df_senseva_pair[(df_senseva_pair['Pos']=='a')&(df_senseva_pair['Merge']=='not-merged')])
notmerge_n=len(df_senseva_pair[(df_senseva_pair['Pos']=='n')&(df_senseva_pair['Merge']=='not-merged')])
notmerge_v=len(df_senseva_pair[(df_senseva_pair['Pos']=='v')&(df_senseva_pair['Merge']=='not-merged')])

merge_a=len(df_senseva_pair[(df_senseva_pair['Pos']=='a')&(df_senseva_pair['Merge']=='merged')])
merge_n=len(df_senseva_pair[(df_senseva_pair['Pos']=='n')&(df_senseva_pair['Merge']=='merged')])
merge_v=len(df_senseva_pair[(df_senseva_pair['Pos']=='v')&(df_senseva_pair['Merge']=='merged')])

rate_a=merge_a/total_a
rate_n=merge_n/total_n
rate_v=merge_v/total_v
print('a',total_a, notmerge_a, merge_a, rate_a)
print('n',total_n, notmerge_n, merge_n, rate_n)
print('v',total_v, notmerge_v, merge_v, rate_v)


a 4 4 0 0.0
n 305 243 62 0.20327868852459016
v 2334 2111 223 0.09554413024850043


In [6]:
#map senseva wn1.7 to wn2.1 
df_pairs=pd.read_csv('Senseval2_sense_pair.csv')

df_wn17wn21=pd.DataFrame(columns=['Pos','sense1_key17','sense1_wn17','sense2_wn17','sense2_key17','sense1_wn21','sense2_wn21','offset1_wn21','offset2_wn21','sense1_wn17_def','sense1_wn21_def','sense2_wn17_def','sense2_wn21_def','Merge'])

#df_pairs_n=df_pairs[df_pairs['Pos']=='n']
#total_pair=len(df_pairs_n)
total_pair=len(df_pairs)
df_senseval2=pd.read_csv('Senseval2_raw.csv')
#df_senseval2[df_senseval2['Synset']=='pull#v#12']['SenseKeys'].values[0]

for i in range(total_pair):
    if (i%1000 ==0):
        print('processing {}/{}'.format(i,total_pair))
    sense1_wn17 = df_pairs.loc[df_pairs.index[i],'Sense1']
    headword=sense1_wn17.split('#')[0]
    sense1_wn17 = sense1_wn17.replace('#','.')
    sense2_wn17 = df_pairs.loc[df_pairs.index[i],'Sense2']
    merge= df_pairs.loc[df_pairs.index[i],'Merge'] 
    sense2_wn17 = sense2_wn17.replace('#','.')
    pos = df_pairs.loc[df_pairs.index[i],'Pos']
    
    lemmas=''
    sense1_wn2=''
    sense1_wn2_def=''
    offset1_wn2=''
    for lemma1 in wn17.synset(sense1_wn17).lemmas():
        if len(sense1_wn2)>2:
            break
        sense1_wn17_def = wn17.synset(sense1_wn17).definition()
        #get sense key for wn21 sense1
        wn17_sense1_key=lemma1.key()
        #enumerate all wn synsests for headword#pos,find the same sense key
        for synset1 in wn2.synsets(headword, pos):
            lemmas = synset1.lemmas()
            if len(sense1_wn2)>2:
                break
            for l1 in lemmas:
                #if sense key equal, keep the wn16 synset name
                if l1.key() == wn17_sense1_key:
                    sense1_wn2=synset1.name()
                    sense1_wn2_def= synset1.definition()
                    offset1_wn2 = str(synset1.offset()).zfill(8) +'-'+ synset1.pos()
                    break

    lemmas=''
    sense2_wn2=''
    offset2_wn2=''
    
    for lemma2 in wn17.synset(sense2_wn17).lemmas():
        if len(sense2_wn2)>2:
            break
        sense2_wn17_def = wn17.synset(sense2_wn17).definition()
        wn17_sense2_key=lemma2.key()
        for synset2 in wn2.synsets(headword, pos):
            lemmas = synset2.lemmas()
            if len(sense2_wn2)>2:
                break
            for l2 in lemmas:
                if l2.key() == wn17_sense2_key:
                    sense2_wn2=synset2.name()
                    sense2_wn2_def= synset2.definition()
                    offset2_wn2 = str(synset2.offset()).zfill(8) +'-'+ synset2.pos()
                    break

    df_wn17wn21=df_wn17wn21.append({'Pos':pos,'sense1_key17':wn17_sense1_key,'sense1_wn17':sense1_wn17,'sense2_wn17':sense2_wn17,'sense2_key17':wn17_sense2_key,'sense1_wn21':sense1_wn2,'sense2_wn21':sense2_wn2,
                                    'offset1_wn21':offset1_wn2,'offset2_wn21':offset2_wn2,'sense1_wn21_def':sense1_wn2_def,'sense2_wn21_def':sense2_wn2_def,'Merge':merge},
                               ignore_index=True)
   
    
df_wn17wn21.to_csv('Senseval_WN17WN21full.csv',index=False)

df_wn17wn21

processing 0/2644
processing 1000/2644
processing 2000/2644


,Pos,sense1_key17,sense1_wn17,sense2_wn17,sense2_key17,sense1_wn21,sense2_wn21,offset1_wn21,offset2_wn21,sense1_wn17_def,sense1_wn21_def,sense2_wn17_def,sense2_wn21_def,Merge
0,v,wander%2:38:00::,wander.v.01,wander.v.03,wander%2:38:02::,roll.v.12,wander.v.03,01863577-v,02083938-v,NaN,move about aimlessly or without any destinatio...,NaN,go via an indirect route or at no set pace,merged
1,v,observe%2:41:02::,observe.v.06,observe.v.09,observe%2:41:04::,observe.v.06,observe.v.09,02553734-v,02554084-v,NaN,"celebrate, as of holidays or rites",NaN,conform one's action or practice to,merged
2,v,observe%2:41:02::,observe.v.06,observe.v.08,observe%2:31:00::,observe.v.06,observe.v.08,02553734-v,00724225-v,NaN,"celebrate, as of holidays or rites",NaN,observe correctly or closely,not-merged
3,v,observe%2:41:04::,observe.v.09,observe.v.08,observe%2:31:00::,observe.v.09,observe.v.08,02554084-v,00724225-v,NaN,conform one's action or practice to,NaN,observe correctly or closely,not-merged
4,v,wash_up%2:35:02::,wash_up.v.03,wash_up.v.01,wash_up%2:29:00::,wash_up.v.03,wash_up.v.01,01520295-v,00024967-v,NaN,wash dishes,NaN,wash one's face and hands,merged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2639,v,draw_up%2:38:01::,draw_up.v.03,draw_up.v.01,draw_up%2:41:00::,draw_up.v.03,draw_up.v.01,01845990-v,02425073-v,NaN,cause (a vehicle) to stop,NaN,"form or arrange in order or formation, as of a...",not-merged
2640,v,draw_up%2:38:01::,draw_up.v.03,draw_up.v.02,draw_up%2:38:02::,draw_up.v.03,draw_up.v.02,01845990-v,01964369-v,NaN,cause (a vehicle) to stop,NaN,straighten oneself,not-merged
2641,v,draw_up%2:41:00::,draw_up.v.01,draw_up.v.02,draw_up%2:38:02::,draw_up.v.01,draw_up.v.02,02425073-v,01964369-v,NaN,"form or arrange in order or formation, as of a...",NaN,straighten oneself,not-merged
2642,v,discover%2:36:00::,discover.v.02,discover.v.04,discover%2:31:00::,discover.v.02,discover.v.04,01623484-v,00713143-v,NaN,"make a discovery, make a new finding",NaN,make a discovery,merged


In [160]:
from collections import Counter
df_wn17wn21['sense12']=df_wn17wn21['sense1_wn21']+'-'+df_wn17wn21['sense2_wn21']

print(len(set(df_wn17wn21['sense12'].to_list())))
Counter(df_wn17wn21['sense12'].to_list())
df_wn17wn21[df_wn17wn21['sense12']=='air.n.06-tune.n.01']


1049


,Pos,sense1_key17,sense1_wn17,sense2_wn17,sense2_key17,sense1_wn21,sense2_wn21,offset1_wn21,offset2_wn21,sense1_wn17_def,sense1_wn21_def,sense2_wn17_def,sense2_wn21_def,Merge,sense12
548,n,air%1:07:00::,air#n#04,tune#n#01,air%1:10:01::,air.n.06,tune.n.01,04671829-n,06932144-n,NaN,a distinctive but intangible quality surroundi...,NaN,a succession of notes forming a distinctive se...,not-merged,air.n.06-tune.n.01
549,n,air%1:07:00::,air#n#04,appeal#n#03,appeal%1:04:00::,air.n.06,tune.n.01,04671829-n,06932144-n,NaN,a distinctive but intangible quality surroundi...,NaN,a succession of notes forming a distinctive se...,not-merged,air.n.06-tune.n.01
550,n,air%1:07:00::,air#n#04,entreaty#n#01,appeal%1:10:00::,air.n.06,tune.n.01,04671829-n,06932144-n,NaN,a distinctive but intangible quality surroundi...,NaN,a succession of notes forming a distinctive se...,not-merged,air.n.06-tune.n.01
551,n,air%1:07:00::,air#n#04,solicitation#n#02,appeal%1:10:02::,air.n.06,tune.n.01,04671829-n,06932144-n,NaN,a distinctive but intangible quality surroundi...,NaN,a succession of notes forming a distinctive se...,not-merged,air.n.06-tune.n.01
552,n,air%1:07:00::,air#n#04,appeal#n#02,appeal%1:07:00::,air.n.06,tune.n.01,04671829-n,06932144-n,NaN,a distinctive but intangible quality surroundi...,NaN,a succession of notes forming a distinctive se...,not-merged,air.n.06-tune.n.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,n,air%1:07:00::,air#n#04,water#n#04,water%1:27:02::,air.n.06,tune.n.01,04671829-n,06932144-n,NaN,a distinctive but intangible quality surroundi...,NaN,a succession of notes forming a distinctive se...,not-merged,air.n.06-tune.n.01
780,n,air%1:07:00::,air#n#04,water_system#n#02,water%1:06:00::,air.n.06,tune.n.01,04671829-n,06932144-n,NaN,a distinctive but intangible quality surroundi...,NaN,a succession of notes forming a distinctive se...,not-merged,air.n.06-tune.n.01
781,n,air%1:07:00::,air#n#04,urine#n#01,water%1:27:01::,air.n.06,tune.n.01,04671829-n,06932144-n,NaN,a distinctive but intangible quality surroundi...,NaN,a succession of notes forming a distinctive se...,not-merged,air.n.06-tune.n.01
782,n,air%1:07:00::,air#n#04,yew#n#02,yew%1:20:00::,air.n.06,tune.n.01,04671829-n,06932144-n,NaN,a distinctive but intangible quality surroundi...,NaN,a succession of notes forming a distinctive se...,not-merged,air.n.06-tune.n.01


In [7]:
#remove sense pair in OntoNotes
features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
df_onetonote_features= pd.read_csv(features_path+'feature_space.csv')
df_otf=df_onetonote_features[['Sense1','Sense2']]
df_otf['se1se2']=df_otf['Sense1']+'-'+df_otf['Sense2']

otf_set=set(df_otf['se1se2'].to_list())

df_senseva_WN21=pd.read_csv('Senseval_WN17WN21full.csv')
df_senseva_WN21['sense1_wn21']=df_senseva_WN21['sense1_wn21'].str.replace('.','#')
df_senseva_WN21['sense2_wn21']=df_senseva_WN21['sense2_wn21'].str.replace('.','#')

df_senseva_WN21['selse2']=df_senseva_WN21['sense1_wn21']+'-'+df_senseva_WN21['sense2_wn21']

senseva_set=set(df_senseva_WN21['selse2'].to_list())

#delete overlap from senseva set


senseva_set - otf_set

df_senseva_WN21=df_senseva_WN21[df_senseva_WN21['selse2'].isin(list(senseva_set - otf_set))]


df_senseva_WN21=df_senseva_WN21[df_senseva_WN21['sense1_wn21'].notna()]
df_senseva_WN21=df_senseva_WN21[df_senseva_WN21['sense2_wn21'].notna()]
df_senseva_WN21.to_csv('Senseval2_sense_pair_W21_nooverlap.csv',index=False)

df_senseva_WN21


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Pos,sense1_key17,sense1_wn17,sense2_wn17,sense2_key17,sense1_wn21,sense2_wn21,offset1_wn21,offset2_wn21,sense1_wn17_def,sense1_wn21_def,sense2_wn17_def,sense2_wn21_def,Merge,selse2
0,v,wander%2:38:00::,wander.v.01,wander.v.03,wander%2:38:02::,roll#v#12,wander#v#03,01863577-v,02083938-v,NaN,move about aimlessly or without any destinatio...,NaN,go via an indirect route or at no set pace,merged,roll#v#12-wander#v#03
1,v,observe%2:41:02::,observe.v.06,observe.v.09,observe%2:41:04::,observe#v#06,observe#v#09,02553734-v,02554084-v,NaN,"celebrate, as of holidays or rites",NaN,conform one's action or practice to,merged,observe#v#06-observe#v#09
2,v,observe%2:41:02::,observe.v.06,observe.v.08,observe%2:31:00::,observe#v#06,observe#v#08,02553734-v,00724225-v,NaN,"celebrate, as of holidays or rites",NaN,observe correctly or closely,not-merged,observe#v#06-observe#v#08
3,v,observe%2:41:04::,observe.v.09,observe.v.08,observe%2:31:00::,observe#v#09,observe#v#08,02554084-v,00724225-v,NaN,conform one's action or practice to,NaN,observe correctly or closely,not-merged,observe#v#09-observe#v#08
4,v,wash_up%2:35:02::,wash_up.v.03,wash_up.v.01,wash_up%2:29:00::,wash_up#v#03,wash_up#v#01,01520295-v,00024967-v,NaN,wash dishes,NaN,wash one's face and hands,merged,wash_up#v#03-wash_up#v#01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2638,v,draw_up%2:38:00::,draw_up.v.05,draw_up.v.02,draw_up%2:38:02::,draw_up#v#05,draw_up#v#02,01845738-v,01964369-v,NaN,come to a halt after driving somewhere,NaN,straighten oneself,not-merged,draw_up#v#05-draw_up#v#02
2639,v,draw_up%2:38:01::,draw_up.v.03,draw_up.v.01,draw_up%2:41:00::,draw_up#v#03,draw_up#v#01,01845990-v,02425073-v,NaN,cause (a vehicle) to stop,NaN,"form or arrange in order or formation, as of a...",not-merged,draw_up#v#03-draw_up#v#01
2640,v,draw_up%2:38:01::,draw_up.v.03,draw_up.v.02,draw_up%2:38:02::,draw_up#v#03,draw_up#v#02,01845990-v,01964369-v,NaN,cause (a vehicle) to stop,NaN,straighten oneself,not-merged,draw_up#v#03-draw_up#v#02
2641,v,draw_up%2:41:00::,draw_up.v.01,draw_up.v.02,draw_up%2:38:02::,draw_up#v#01,draw_up#v#02,02425073-v,01964369-v,NaN,"form or arrange in order or formation, as of a...",NaN,straighten oneself,not-merged,draw_up#v#01-draw_up#v#02


In [8]:
#analysis of paris after removing overlap
df_senseva_pair=df_senseva_WN21[['Pos','sense1_wn21','sense2_wn21','Merge']]
total_a=len(df_senseva_pair[df_senseva_pair['Pos']=='a'])
total_n=len(df_senseva_pair[df_senseva_pair['Pos']=='n'])
total_v=len(df_senseva_pair[df_senseva_pair['Pos']=='v'])

notmerge_a=len(df_senseva_pair[(df_senseva_pair['Pos']=='a')&(df_senseva_pair['Merge']=='not-merged')])
notmerge_n=len(df_senseva_pair[(df_senseva_pair['Pos']=='n')&(df_senseva_pair['Merge']=='not-merged')])
notmerge_v=len(df_senseva_pair[(df_senseva_pair['Pos']=='v')&(df_senseva_pair['Merge']=='not-merged')])

merge_a=len(df_senseva_pair[(df_senseva_pair['Pos']=='a')&(df_senseva_pair['Merge']=='merged')])
merge_n=len(df_senseva_pair[(df_senseva_pair['Pos']=='n')&(df_senseva_pair['Merge']=='merged')])
merge_v=len(df_senseva_pair[(df_senseva_pair['Pos']=='v')&(df_senseva_pair['Merge']=='merged')])

rate_a=merge_a/total_a
rate_n=merge_n/total_n
rate_v=merge_v/total_v
print('a',total_a, notmerge_a, merge_a, rate_a)
print('n',total_n, notmerge_n, merge_n, rate_n)
print('v',total_v, notmerge_v, merge_v, rate_v)

a 2 2 0 0.0
n 305 243 62 0.20327868852459016
v 2115 1916 199 0.09408983451536643


In [12]:
#get features using perl language
df_senseva_pair21=df_senseva_WN21[['Pos','sense1_wn21','sense2_wn21','Merge']]
df_senseva_pair21.columns=['Pos','Sense1','Sense2','Merge']
df_senseva_pair21.to_csv('Senseva2_sensepairs_wn21.csv')

In [13]:
'''
Build feature space:
1, twin: the number of shared synonyms between two synsets
2, antonym: whether two synsets share an antonym
3, pertainym: whether two synsets share an pertainym
4, deriv: whether two synsets share derivationally related forms
5, verbgrp: whether two verb synsets are linked in a VERBGROUP (indicating semantic similarity)
6, verbfrm: whether two verb synsets share a VERBFRAM (indicating syntactic similarity) 

'''
#wordnet 2.1
df_pairs=pd.read_csv('Senseva2_sensepairs_wn21.csv')
#df_pairs
for i in range(len(df_pairs)):
    pertainyms1=[]
    antonyms1=[]
    deriv1=[]
    lemmas1=[]
    verbgroup1=[]
    verbframe1=[]
    pertainyms2=[]
    antonyms2=[]
    deriv2=[]
    lemmas2=[]
    verbgroup2=[]
    verbframe2=[]
    pertainymsflag=''
    antonymsflag=''
    derivflag=''
    lemmasflag=''
    verbgroupflag=''
    verbframeflag=''
    hyper_min=''
    hyper_max=''

    sense1 = df_pairs.loc[df_pairs.index[i],'Sense1']
    sense1 = sense1.replace('#','.')
    sense2 = df_pairs.loc[df_pairs.index[i],'Sense2']
    sense2 = sense2.replace('#','.')
    pos = df_pairs.loc[df_pairs.index[i],'Pos']
    
    # sense1's twin, pertainyms, antonyms, derivationally_related_forms,
    # verb group, frame id, hyper_min, hyper_max
    try:
        
        #WN, WNMax feature
        #calculate hyper distance
        sense1_hyper = wn2.synset(sense1)
        sense1_hypers = lambda s: s.hypernyms()
        hyper1= list(sense1_hyper.closure(sense1_hypers))
        
        sense2_hyper = wn2.synset(sense2)
        sense2_hypers = lambda s: s.hypernyms()
        hyper2 = list(sense2_hyper.closure(sense2_hypers))
           
        #find the nearest hyper, and average the distance as the least distance
        for h1 in hyper1:
            if h1 in hyper2:
                hyper_min = (hyper1.index(h1)+hyper2.index(h1))/2
                break
        


        #find the farest hyper, and average the distance as the largest distance
        for h1 in reversed(hyper1):
            if h1 in hyper2:
                hyper_max = (hyper1.index(h1)+hyper2.index(h1))/2
                break
                
        
        for lemma in wn2.synset(sense1).lemmas():
            
            pertainyms1=pertainyms1+lemma.pertainyms()
            antonyms1=antonyms1+lemma.antonyms()
            deriv1=deriv1+lemma.derivationally_related_forms()
            lemmas1.append(lemma.name())
            
            if pos=='v':
                verbgroup1=verbgroup1+lemma.verb_groups()
                verbframe1=verbframe1+lemma.frame_ids()

                
        # sense2's pertainyms, antonyms, derivationally_related_forms
        for lemma in wn2.synset(sense2).lemmas():
            
            pertainyms2=pertainyms2+lemma.pertainyms()
            antonyms2=antonyms2+lemma.antonyms()
            deriv2=deriv2+lemma.derivationally_related_forms()
            lemmas2.append(lemma.name())
             
            #verb group, verb frame
            if pos=='v':
                verbgroup2=verbgroup2+lemma.verb_groups()
                verbframe2=verbframe2+lemma.frame_ids()
                
      
        if (len(set(pertainyms1)&set(pertainyms2))>0):
            pertainymsflag=1
        else:
            pertainymsflag=0

        if (len(set(antonyms1)&set(antonyms2))>0):
            antonymsflag=1
        else:
            antonymsflag=0 

        if (len(set(deriv1)&set(deriv2))>0):
            derivflag=1
        else:
            derivflag=0 
        
        #verb group, verb frame
        if pos=='v':
            if (len(set(verbgroup1)&set(verbgroup2))>0):
                verbgroupflag=1
            else:
                verbgroupflag=0
                
            if (len(set(verbframe1)&set(verbframe2))>0):
                verbframeflag=1
            else:
                verbframeflag=0
            

        
        lemmasflag=len(set(lemmas1)&set(lemmas2))
       
        
        df_pairs.loc[df_pairs.index[i],'pertainyms']=pertainymsflag
        df_pairs.loc[df_pairs.index[i],'antonyms']=antonymsflag
        df_pairs.loc[df_pairs.index[i],'deriv']=derivflag
        df_pairs.loc[df_pairs.index[i],'lemmas']=lemmasflag
        
        df_pairs.loc[df_pairs.index[i],'verbgroup']=verbgroupflag
        df_pairs.loc[df_pairs.index[i],'verbframe']=verbframeflag
         
        df_pairs.loc[df_pairs.index[i],'hyper_min']=hyper_min
        df_pairs.loc[df_pairs.index[i],'hyper_max']=hyper_max

        
    except:
        None
    
df_pairs.to_csv('Senseva2_SensesPairs_WNFeatures.csv')    
 

In [15]:
#create feature based on mapping between WN and OED, the mapping is based on WN21
#read mapping file
file_map_wnoed=open('/Users/gary/Documents/2020Fall/IntroNLP/project/FeatureSpace/sense_clusters-21.senses','r')

#read sysnset pairs
df_pairs=pd.read_csv('Senseva2_sensepairs_wn21.csv')


#df_pairs_n=df_pairs[df_pairs['Pos']=='n']
#total_pair=len(df_pairs_n)
total_pair=len(df_pairs)
for i in range(total_pair):
    if (i%1000 ==0):
        print('processing {}/{}'.format(i,total_pair))
    sense1_wn21 = df_pairs.loc[df_pairs.index[i],'Sense1']
    headword=sense1_wn21.split('#')[0]
    sense1_wn21 = sense1_wn21.replace('#','.')
    sense2_wn21 = df_pairs.loc[df_pairs.index[i],'Sense2']
    sense2_wn21 = sense2_wn21.replace('#','.')
    pos = df_pairs.loc[df_pairs.index[i],'Pos']
    
    sense1_keys=''
    sense2_keys=''
    try:
        for lemma1 in wn2.synset(sense1_wn21).lemmas():
            #get sense key for wn21 sense1
            sense1_keys=sense1_keys+lemma1.key()+';'
        for lemma2 in wn2.synset(sense2_wn21).lemmas():
            #get sense key for wn21 sense1
            sense2_keys=sense2_keys+lemma1.key()+';'
    except:
        None

    df_pairs.loc[df_pairs.index[i],'Sense1_keys']=sense1_keys
    df_pairs.loc[df_pairs.index[i],'Sense2_keys']=sense2_keys

#filter sense keys is null
df_pairs = df_pairs[(df_pairs['Sense1_keys'].str.len()>5)&(df_pairs['Sense1_keys'].str.len()>5)]

#get sense clusterings from the mapping between WN and OED
lines = file_map_wnoed.readlines()
count=0
wnoed_lst=[] # a list of sets.
for line in lines:
    count+=1
    line=line.strip('\n')
    if ' ' in line:
        wnoed_lst.append(set(line.split(' ')))

for i in range(len(df_pairs)):
    if i%1000 ==0:
        print('processing {}/{}'.format(i,total_pair))
    sense1_key=df_pairs.loc[df_pairs.index[i],'Sense1_keys']
    sk1_set=set(sense1_key.split(';'))
    
    sense2_key=df_pairs.loc[df_pairs.index[i],'Sense2_keys']
    sk2_set=set(sense2_key.split(';'))
    
    feature=0
    for  w in wnoed_lst:
        sk1_len=len(sk1_set & w)
        sk2_len=len(sk2_set & w)
        if (sk1_len>0 and sk2_len>0):
            feature=1
            break
    
    df_pairs.loc[df_pairs.index[i],'wn_oed_feature']=feature
    
df_pairs.to_csv('WN_OED_Map_Feature_Senseva2_SensesPairs.csv',index=False)


processing 0/2423
processing 1000/2423
processing 2000/2423
processing 0/2423
processing 1000/2423
processing 2000/2423


In [25]:
#concatnate all the features together.

features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
feature_files=['Senseva2_sensepairs_wn21.csv','Senseva2_pairs_wnsimilarity.csv','Senseva2_SensesPairs_WNFeatures.csv','WN_OED_Map_Feature_Senseva2_SensesPairs.csv']


df_sensepari=pd.read_csv(features_path+feature_files[0])
df_sensepari= df_sensepari[['Pos', 'Sense1', 'Sense2', 'Merge']]
df_sensepari=df_sensepari.drop_duplicates()

print('df_sensepari',len(df_sensepari))
df_wnpackage=pd.read_csv(features_path+feature_files[1])
df_wnpackage=df_wnpackage.drop_duplicates()
print('df_wnpackage',len(df_wnpackage))

df_features_tmp = df_sensepari.merge(df_wnpackage,left_on=['Sense1', 'Sense2','Merge'], 
                              right_on=['sense1', 'sense2','merge'], how='left')


df_features_tmp=df_features_tmp[['Pos', 'Sense1', 'Sense2', 'Merge', 
       'lch', 'hso', 'jcn', 'leskvalue', 'linvalue', 'resvalue', 'vecvalue',
       'wupvalue']]
df_features_tmp=df_features_tmp.drop_duplicates()
print('df_features_tmp',len(df_features_tmp))

df_wncorpus=pd.read_csv(features_path+feature_files[2])

print('df_wncorpus',len(df_wncorpus))

df_features_tmp1 = df_features_tmp.merge(df_wncorpus,left_on=['Pos', 'Sense1', 'Sense2', 'Merge'], 
                              right_on=['Pos', 'Sense1', 'Sense2', 'Merge'], how='left')

df_features_tmp1.columns
print('df_features_tmp1',len(df_features_tmp1))


df_features_tmp1=df_features_tmp1[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 
        'pertainyms', 'antonyms', 'deriv', 'lemmas',
       'verbgroup', 'verbframe', 'hyper_min', 'hyper_max']]

df_features_tmp1=df_features_tmp1.drop_duplicates() 

#wn-oed mapping feature
df_wnoedfea=pd.read_csv(features_path+feature_files[3])
df_wnoedfea
print('df_wnoedfea',len(df_wnoedfea))
df_features = df_features_tmp1.merge(df_wnoedfea[['Pos', 'Sense1', 'Sense2', 'Merge','wn_oed_feature']],left_on=['Pos', 'Sense1', 'Sense2','Merge'], 
                              right_on=['Pos', 'Sense1', 'Sense2', 'Merge'], how='left')

df_features=df_features[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 'pertainyms',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe', 'hyper_min',
       'hyper_max', 'wn_oed_feature']]

df_features=df_features.drop_duplicates()
df_features
print('df_features',len(df_features))
df_features.to_csv('senseval2_feature_space.csv',index=False)


df_sensepari 2423
df_wnpackage 2464
df_features_tmp 2423
df_wncorpus 2423
df_features_tmp1 2423
df_wnoedfea 2423
df_features 2423


In [26]:
#analyze the feature space
features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
df_features= pd.read_csv(features_path+'senseval2_feature_space.csv')
#remove wrong senses (no sense number)
df_features = df_features[df_features['Sense1'].str.contains('[0-9]')]
df_features = df_features[df_features['Sense2'].str.contains('[0-9]')]

#df_features[~(df_features['Sense1'].str.contains('[0-9]'))]
#df_features[~(df_features['Sense2'].str.contains('[0-9]'))]

#deal with NaN columns
#remove 'lch' is NaN rows, as all other features are also NaN
df_features = df_features[~(df_features['lch'].isna())]
df_features


,Pos,Sense1,Sense2,Merge,lch,hso,jcn,leskvalue,linvalue,resvalue,...,wupvalue,pertainyms,antonyms,deriv,lemmas,verbgroup,verbframe,hyper_min,hyper_max,wn_oed_feature
0,v,roll#v#12,wander#v#03,merged,1.945910,4.0,0.000000,10.0,0.0,3.318246,...,0.571429,0.0,0.0,0.0,1.0,0.0,1.0,0.5,0.5,0.0
1,v,observe#v#06,observe#v#09,merged,2.233592,0.0,0.056615,9.0,0.0,0.000000,...,0.500000,0.0,0.0,1.0,2.0,0.0,1.0,NaN,NaN,1.0
2,v,observe#v#06,observe#v#08,not-merged,2.233592,0.0,0.056311,11.0,0.0,0.000000,...,0.500000,0.0,0.0,0.0,2.0,0.0,1.0,NaN,NaN,1.0
3,v,observe#v#09,observe#v#08,not-merged,2.233592,0.0,0.053396,9.0,0.0,0.000000,...,0.500000,0.0,0.0,0.0,2.0,0.0,1.0,NaN,NaN,1.0
4,v,wash_up#v#03,wash_up#v#01,merged,1.386294,2.0,0.000000,5.0,0.0,0.000000,...,0.250000,0.0,0.0,1.0,1.0,0.0,1.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2418,v,draw_up#v#05,draw_up#v#02,not-merged,1.386294,0.0,0.051186,4.0,0.0,0.000000,...,0.250000,0.0,0.0,0.0,2.0,0.0,0.0,NaN,NaN,0.0
2419,v,draw_up#v#03,draw_up#v#01,not-merged,1.540445,0.0,0.043985,3.0,0.0,0.000000,...,0.285714,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,0.0
2420,v,draw_up#v#03,draw_up#v#02,not-merged,1.386294,0.0,0.046550,4.0,0.0,0.000000,...,0.250000,0.0,0.0,0.0,2.0,0.0,0.0,NaN,NaN,0.0
2421,v,draw_up#v#01,draw_up#v#02,not-merged,1.252763,0.0,0.048102,2.0,0.0,0.000000,...,0.222222,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,0.0


In [113]:
#for each PoS, split the features into a training set (70%) and a testing set (30%)
# I remove wn_domain_feature,as it is same (1) for all pairs
# I remove 'hyper_min', 'hyper_max', 'topic_similarity' as they have NaN values 
# I remove 'pertainyms' as it is for adj.
features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'
df_features1= pd.read_csv(features_path+'senseval2_feature_space_sf1.csv')

df_features3= pd.read_csv(features_path+'senseval2_feature_space.csv')

df_features=pd.concat([df_features1,df_features3])
#df_features= pd.read_csv(features_path+'senseval2_feature_space.csv')



df_features = df_features[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn',
       'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'pertainyms', 'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe',
       'hyper_min', 'hyper_max', 'wn_oed_feature']]
df_features_n = df_features[df_features['Pos']=='n'][['Pos', 'Sense1', 'Sense2', 'Merge', 
        'lch', 'hso', 'jcn', 'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas','wn_oed_feature']]
df_features_v = df_features[df_features['Pos']=='v'][['Pos', 'Sense1', 'Sense2', 'Merge', 
       'lch', 'hso', 'jcn','leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe','wn_oed_feature']]

#replace labels with 0 or 1
df_features_v=df_features_v.replace(['not-merged','merged'],[0,1])
v_nm=len(df_features_v[df_features_v['Merge']==0])
v_m=len(df_features_v[df_features_v['Merge']==1])
print('v->total:{}, not_merged:{}, merged:{},merge rate:{}'.format(v_nm+v_m,v_nm,v_m,v_m/(v_nm+v_m)))

df_features_n=df_features_n.replace(['not-merged','merged'],[0,1])
n_nm=len(df_features_n[df_features_n['Merge']==0])
n_m=len(df_features_n[df_features_n['Merge']==1])
print('n->total:{},not_merged:{}, merged:{},merge rate:{}'.format(n_nm+n_m, n_nm,n_m,n_m/(n_nm+n_m)))

v->total:2827, not_merged:2613, merged:214,merge rate:0.07569862044570216
n->total:432,not_merged:347, merged:85,merge rate:0.19675925925925927


In [ ]:
print(v_m/)

In [105]:
#check if NaN exist
df_features_n.columns[df_features_n.isna().any()].to_list()


[]

In [106]:

df_features


,Pos,Sense1,Sense2,Merge,lch,hso,jcn,leskvalue,linvalue,resvalue,...,wupvalue,pertainyms,antonyms,deriv,lemmas,verbgroup,verbframe,hyper_min,hyper_max,wn_oed_feature
0,v,roll#v#12,wander#v#03,not-merged,1.945910,4.0,0.000000,10.0,0.0,3.318246,...,0.571429,0.0,0.0,0.0,1.0,0.0,1.0,0.5,0.5,0.0
1,v,observe#v#08,observe#v#09,not-merged,2.233592,0.0,0.053396,9.0,0.0,0.000000,...,0.500000,0.0,0.0,0.0,2.0,0.0,1.0,NaN,NaN,0.0
2,v,draw#v#19,draw#v#07,not-merged,1.540445,0.0,0.047931,8.0,0.0,0.000000,...,0.285714,0.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN,1.0
3,v,draw#v#19,draw#v#10,not-merged,1.386294,0.0,0.045832,2.0,0.0,0.000000,...,0.250000,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,1.0
4,v,draw#v#19,draw#v#05,not-merged,1.540445,0.0,0.048413,5.0,0.0,0.000000,...,0.285714,0.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2418,v,draw_up#v#05,draw_up#v#02,not-merged,1.386294,0.0,0.051186,4.0,0.0,0.000000,...,0.250000,0.0,0.0,0.0,2.0,0.0,0.0,NaN,NaN,0.0
2419,v,draw_up#v#03,draw_up#v#01,not-merged,1.540445,0.0,0.043985,3.0,0.0,0.000000,...,0.285714,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,0.0
2420,v,draw_up#v#03,draw_up#v#02,not-merged,1.386294,0.0,0.046550,4.0,0.0,0.000000,...,0.250000,0.0,0.0,0.0,2.0,0.0,0.0,NaN,NaN,0.0
2421,v,draw_up#v#01,draw_up#v#02,not-merged,1.252763,0.0,0.048102,2.0,0.0,0.000000,...,0.222222,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,0.0


In [26]:
#apply SVM for Verb F1=0.4989
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,StratifiedKFold,LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
import numpy as np

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

def split_data(X,Y,rate):
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=rate,random_state=42)
    return X_train,X_test,Y_train,Y_test

def cross_validation(X_train, Y_train, C_list, K):
    for c in C_list:
        clf1 = SVC(kernel='linear',C=c).fit(X_train,Y_train)

        strat_scores = cross_val_score(clf1,X_train,Y_train,cv=StratifiedKFold(K,random_state=10,shuffle=True))
        print('C=',c)
        print("The Stratified Cross Validation Score :"+str(strat_scores))
        print("The Average Stratified Cross Validation Score :"+str(strat_scores.mean()))

def predict(X_train, Y_train, X_test, c):
    
    clf1 = SVC(kernel='rbf', C=c).fit(X_train,Y_train)
    #print('features important:')
    #f_importances(clf1.coef_, features_names)
    print("shape of Y :"+str(Y_train.shape))
    print("shape of X :"+str(X_train.shape))
    y_pred = clf1.predict(X_test)
    return clf1, y_pred

def cal_FScore(y_true, y_pred):
    F1=f1_score(y_true, y_pred, average='macro')
    return F1

def Fscore(df_pos, tune_flag, pos, c=None):
    '''
    tune_flag: Y-> tune c
    '''
    Y=np.array(df_pos['Merge'])
    X=df_pos.drop(['Pos','Sense1','Sense2','Merge'],axis=1)
    X=np.array(X)
    features_names=['1','2','3','4','5','6','7','8','9','10','11','12','13','14']
    print("shape of Y :"+str(Y.shape))
    print("shape of X :"+str(X.shape))
    
    #normalize data, Snow did not mention this
    #scaler = MinMaxScaler()
    #X = scaler.fit_transform(X)
    #x_idex=X
    x_idex=0
    #split data
    rate=0.3
    X_train,X_test,Y_train,Y_test = split_data(X,Y,rate)
    if tune_flag=='Y':
        C_list=[1e2,1e3,1e4,1e5,1e6,1e7]
        K=10
        cross_validation(X_train, Y_train, C_list, K)
    else:
        #predict
        clf1, y_pred= predict(X_train, Y_train, X_test, c)
        print('y_pred sum:',np.sum(y_pred))
        print('Y_train sum:',np.sum(Y_train))
        #F1 score
        F1= cal_FScore(Y_test,y_pred)
        print('{}:{}'.format(pos,F1))
        #base line

        y_pre_baseline=np.ones(len(Y_test))
        F1_base= cal_FScore(Y_test,y_pre_baseline)
        print('Baseline:',F1_base)
        return F1, X_test, Y_test, y_pred, x_idex



''' 

Y_v = np.array(df_features_v['Merge'])
X_v = df_features_v.drop(['Pos','Sense1','Sense2','Merge'],axis=1)
X_v = np.array(X_v)
features_names=['1','2','3','4','5','6','7','8','9','10','11','12','13','14']
print("shape of Y :"+str(Y_v.shape))
print("shape of X :"+str(X_v.shape))

#normalize data, Snow did not mention this
scaler = MinMaxScaler()
X_v = scaler.fit_transform(X_v)

#split data
rate=0.3
X_train,X_test,Y_train,Y_test = split_data(X_v,Y_v,rate)

#cross validation c=3 average score=0.75233236151603
C_list=[1,2,3,5,10]
K=10
#cross_validation(X_train, Y_train, C_list, K)
c=3

#predict
clf1, y_pred= predict(X_train, Y_train, X_test, c)

#F1 score
F1= cal_FScore(Y_test,y_pred)

print('Verb:',F1)
#base line
y_pre_baseline_v=np.ones(len(Y_test))
F1= cal_FScore(Y_test,y_pre_baseline_v)

print('Baseline:',F1)
''' 
#for each PoS, split the features into a training set (70%) and a testing set (30%)
# I remove wn_domain_feature,as it is same (1) for all pairs
# I remove 'hyper_min', 'hyper_max', 'topic_similarity' as they have NaN values 
# I remove 'pertainyms' as it is for adj.
features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'

#combine OntoNotes and SENSEVAL-2
df_features1= pd.read_csv(features_path+'feature_space.csv')

df_features4= pd.read_csv(features_path+'senseval2_feature_space.csv')

df_features3= pd.read_csv(features_path+'senseval2_feature_space_sf1.csv')

df_features2=pd.concat([df_features4,df_features3])


df_features1 = df_features1[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn',
       'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'pertainyms', 'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe',
       'hyper_min', 'hyper_max', 'wn_oed_feature']]

df_features2 = df_features2[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn',
       'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'pertainyms', 'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe',
       'hyper_min', 'hyper_max', 'wn_oed_feature']]

df_features = pd.concat([df_features1,df_features2])

df_features_n = df_features[df_features['Pos']=='n'][['Pos', 'Sense1', 'Sense2', 'Merge', 
        'lch', 'hso', 'jcn', 'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas','wn_oed_feature']]
df_features_v = df_features[df_features['Pos']=='v'][['Pos', 'Sense1', 'Sense2', 'Merge', 
       'lch', 'hso', 'jcn','leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe','wn_oed_feature']]

#replace labels with 0 or 1
df_features_v=df_features_v.replace(['not-merged','merged'],[0,1])
v_nm=len(df_features_v[df_features_v['Merge']==0])
v_m=len(df_features_v[df_features_v['Merge']==1])
print('v->not_merged:{}, merged:{}'.format(v_nm,v_m))

df_features_n=df_features_n.replace(['not-merged','merged'],[0,1])
n_nm=len(df_features_n[df_features_n['Merge']==0])
n_m=len(df_features_n[df_features_n['Merge']==1])
print('n->not_merged:{}, merged:{}'.format(n_nm,n_m))
#df_features_v

#Fscore(df_pos=df_features_v, tune_flag='Y', c=None)
c=10
F1_v,x_test_v, y_test_v, y_pred_v,x_idex_v=Fscore(df_pos=df_features_v, tune_flag='N', c=c,pos='verb')

F1_n,x_test_n, y_trest_n, y_pred_n,x_idex_n=Fscore(df_pos=df_features_n, tune_flag='N', c=c, pos='noun')

print('n:',F1_n,'v:',F1_v)


v->not_merged:9876, merged:2751
n->not_merged:2596, merged:367
shape of Y :(12627,)
shape of X :(12627, 14)
shape of Y :(8838,)
shape of X :(8838, 14)
y_pred sum: 0
Y_train sum: 1900
verb:0.436747435706853
Baseline: 0.18340517241379312
shape of Y :(2963,)
shape of X :(2963, 12)
shape of Y :(2074,)
shape of X :(2074, 12)
y_pred sum: 5
Y_train sum: 258
noun:0.4751180836707153
Baseline: 0.1092184368737475
n: 0.4751180836707153 v: 0.436747435706853


In [48]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,StratifiedKFold,LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score

def split_data(X,Y,rate):
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=rate,random_state=42)
    return X_train,X_test,Y_train,Y_test

def cal_FScore(y_true, y_pred):
    F1=f1_score(y_true, y_pred, average='macro')
    return F1

features_path='/Users/gary/Documents/2020Fall/IntroNLP/project/'

#combine OntoNotes and SENSEVAL-2
df_features1= pd.read_csv(features_path+'feature_space.csv')

df_features4= pd.read_csv(features_path+'senseval2_feature_space.csv')

df_features3= pd.read_csv(features_path+'senseval2_feature_space_sf1.csv')

df_features2=pd.concat([df_features4,df_features3])


df_features1 = df_features1[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn',
       'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'pertainyms', 'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe',
       'hyper_min', 'hyper_max', 'wn_oed_feature']]

df_features2 = df_features2[['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn',
       'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'pertainyms', 'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe',
       'hyper_min', 'hyper_max', 'wn_oed_feature']]

df_features = pd.concat([df_features1,df_features2])

df_features_n = df_features[df_features['Pos']=='n'][['Pos', 'Sense1', 'Sense2', 'Merge', 
        'lch', 'hso', 'jcn', 'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas','wn_oed_feature']]
df_features_v = df_features[df_features['Pos']=='v'][['Pos', 'Sense1', 'Sense2', 'Merge', 
       'lch', 'hso', 'jcn','leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe','wn_oed_feature']]

#replace labels with 0 or 1
df_features_v=df_features_v.replace(['not-merged','merged'],[0,1])
v_nm=len(df_features_v[df_features_v['Merge']==0])
v_m=len(df_features_v[df_features_v['Merge']==1])
print('v->not_merged:{}, merged:{}'.format(v_nm,v_m))

df_features_n=df_features_n.replace(['not-merged','merged'],[0,1])
n_nm=len(df_features_n[df_features_n['Merge']==0])
n_m=len(df_features_n[df_features_n['Merge']==1])
print('n->not_merged:{}, merged:{}'.format(n_nm,n_m))

df_pos=df_features_v

Y=np.array(df_pos['Merge'])
X=df_pos.drop(['Pos','Sense1','Sense2','Merge'],axis=1)
X=np.array(X)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train,X_test,Y_train,Y_test = split_data(X,Y,0.3)

#train
clf1 = SVC(kernel='rbf', C=10).fit(X_train,Y_train)

y_pred = clf1.predict(X_test)
cal_FScore(Y_test, y_pred)


v->not_merged:9876, merged:2751
n->not_merged:2596, merged:367


0.5327393941863595

In [21]:
y_pred = clf1.predict(X_test1)
cal_FScore(Y_test1, y_pred) 

0.4692537313432836

In [45]:
len(Y_test)

# scikit-learn bootstrap
from sklearn.utils import resample
# data sample
data = list(range(len(Y_test)))
# prepare bootstrap sample

diff=[]
alpha=2*(0.5174-0.4228)
#alpha=2*(0.5327-0.4319)
snow=0.4228
for i in range(3000):
    boot = resample(data, replace=True, n_samples=len(Y_test))

    X_test1 = []
    Y_test1=[]
    for i in boot:
        X_test1.append(X_test[i])
        Y_test1.append(Y_test[i])

    y_pred = clf1.predict(X_test1)
    f1=cal_FScore(Y_test1, y_pred) 
    diff.append(f1-snow)
s=0
for j in diff:
    if j>(alpha):
        s+=1
print('p:',s/30)
        


p: 0.0


In [49]:

len(Y_test)

3789

In [53]:
df_features_v.to_csv(features_path+'feature_space_v.csv')

In [148]:
#error data analysis
#output index for nouns error data
differ_lst_n =[]
for i in range(len(y_pred_n)):
    if y_pred_n[i] != y_trest_n[i]:
        differ_lst_n.append(i)
differ_lst_n

for j in range(len(x_idex_n)):
    if np.array_equal(x_idex_n[j],x_test_n[888]):
        n_idx=j
print(df_features_n[n_idx:n_idx+1])

#output index for verbs error data
differ_lst_v =[]
for i in range(len(y_pred_v)):
    if y_pred_v[i] != y_test_v[i]:
        differ_lst_v.append(i)
differ_lst_v

for j in range(len(x_idex_v)):
    if np.array_equal(x_idex_v[j],x_test_v[12]):
        v_idx=j
        break

print(df_features_v[v_idx:v_idx+1])

     Pos     Sense1     Sense2  Merge       lch  hso       jcn  leskvalue  \
3089   n  share#n#2  share#n#4      1  1.239691  0.0  0.071893       28.0   

      linvalue  resvalue  vecvalue  wupvalue  antonyms  deriv  lemmas  \
3089  0.069092  0.516189  0.773146  0.444444       0.0    0.0     2.0   

      wn_oed_feature  
3089             1.0  
     Pos      Sense1      Sense2  Merge       lch  hso       jcn  leskvalue  \
5875   v  retain#v#1  retain#v#2      0  1.252763  0.0  0.049058        7.0   

      linvalue  resvalue  vecvalue  wupvalue  antonyms  deriv  lemmas  \
5875       0.0       0.0  0.539534  0.222222       0.0    0.0     1.0   

      verbgroup  verbframe  wn_oed_feature  
5875        0.0        0.0             0.0  


In [105]:
print(df_features_n[2601:2602])

    Pos       Sense1       Sense2  Merge       lch  hso       jcn  leskvalue  \
677   n  extent#n#02  extent#n#01      1  1.691676  0.0  0.086012       51.0   

     linvalue  resvalue  vecvalue  wupvalue  antonyms  deriv  lemmas  \
677  0.291538   2.39217  0.575617     0.625       0.0    1.0     1.0   

     wn_oed_feature  
677             0.0  


In [147]:
#x_test_v[8]
for d in differ_lst_n:
    if y_pred_n[d]==1:
        None
    else:
        print(d)

9
12
22
23
33
38
58
59
72
73
79
90
95
102
113
136
139
143
144
149
179
180
183
184
200
216
232
238
241
249
255
272
275
281
294
296
301
321
332
348
350
362
367
380
399
401
404
416
423
442
452
460
473
480
485
496
502
522
533
535
552
553
559
609
618
619
622
624
634
648
649
654
657
659
660
677
683
686
698
703
711
714
715
723
731
732
742
745
747
749
779
799
800
802
805
806
809
824
855
863
869
877
888


In [149]:
#df_features_v[(df_features_v['Sense1'].str.contains('find')) & (df_features_v['Merge']==1)]

df_features_n[df_features_n['Sense1'].str.contains('share')]




,Pos,Sense1,Sense2,Merge,lch,hso,jcn,leskvalue,linvalue,resvalue,vecvalue,wupvalue,antonyms,deriv,lemmas,wn_oed_feature
3081,n,share#n#1,share#n#5,0,0.502092,0.0,0.054223,19.0,0.000000,0.000000,0.351156,0.153846,0.0,0.0,1.0,0.0
3082,n,share#n#1,share#n#2,0,1.072637,0.0,0.082931,52.0,0.078864,0.516189,0.421149,0.400000,0.0,0.0,1.0,0.0
3083,n,share#n#1,share#n#3,0,0.929536,0.0,0.075192,23.0,0.072035,0.516189,0.340837,0.363636,0.0,0.0,1.0,0.0
3084,n,share#n#1,share#n#4,0,0.929536,0.0,0.071123,27.0,0.068403,0.516189,0.325394,0.363636,0.0,0.0,1.0,0.0
3085,n,share#n#5,share#n#2,0,0.693147,0.0,0.054669,36.0,0.000000,0.000000,0.857963,0.181818,0.0,0.0,1.0,0.0
3086,n,share#n#5,share#n#3,0,0.593064,0.0,0.051196,11.0,0.000000,0.000000,0.534629,0.166667,0.0,0.0,1.0,0.0
3087,n,share#n#5,share#n#4,0,0.593064,0.0,0.049276,11.0,0.000000,0.000000,0.722776,0.166667,0.0,0.0,1.0,0.0
3088,n,share#n#2,share#n#3,1,1.239691,0.0,0.076053,26.0,0.072800,0.516189,0.602785,0.444444,0.0,1.0,2.0,1.0
3089,n,share#n#2,share#n#4,1,1.239691,0.0,0.071893,28.0,0.069092,0.516189,0.773146,0.444444,0.0,0.0,2.0,1.0
3090,n,share#n#3,share#n#4,1,1.691676,0.0,0.082918,18.0,0.254765,2.061433,0.495092,0.700000,0.0,0.0,1.0,1.0


In [151]:
#from nltk.corpus import wordnet as wn
wn2.synset('share.n.4').definition()


'any one of a number of individual efforts in a common endeavor'

In [23]:
#nouns features analysis
n_columns=['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 'antonyms', 'deriv',
       'lemmas', 'wn_oed_feature']
#n_columns.remove('lch')
df_features_n = df_features[df_features['Pos']=='n'][['Pos', 'Sense1', 'Sense2', 'Merge', 
        'lch', 'hso', 'jcn', 'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas','wn_oed_feature']]

for feature in n_columns[4:]:
    print('feature:',feature)
    pru_col=['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 'antonyms', 'deriv',
       'lemmas', 'wn_oed_feature']
    df_features_n = df_features[df_features['Pos']=='n'][['Pos', 'Sense1', 'Sense2', 'Merge', 
        'lch', 'hso', 'jcn', 'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas','wn_oed_feature']]
    pru_col.remove(feature)
    df_features_n = df_features_n[pru_col]
    df_features_n=df_features_n.replace(['not-merged','merged'],[0,1])
    n_nm=len(df_features_n[df_features_n['Merge']==0])
    n_m=len(df_features_n[df_features_n['Merge']==1])
    print('n->not_merged:{}, merged:{}'.format(n_nm,n_m))
    #df_features_v

    #Fscore(df_pos=df_features_v, tune_flag='Y', c=None)
    #c=1
    #Fscore(df_pos=df_features_v, tune_flag='N', c=1,pos='verb')
    c=10
    F1_n_c=Fscore(df_pos=df_features_n, tune_flag='N', c=c, pos='noun')
    print('diff:',F1_n-F1_n_c)
    

feature: lch
n->not_merged:2596, merged:367
shape of Y :(2963,)
shape of X :(2963, 11)
shape of Y :(2074,)
shape of X :(2074, 11)
y_pred sum: 12
Y_train sum: 258
noun:0.5166960104141209
Baseline: 0.1092184368737475
diff: 0.0007346289103663528
feature: hso
n->not_merged:2596, merged:367
shape of Y :(2963,)
shape of X :(2963, 11)
shape of Y :(2074,)
shape of X :(2074, 11)
y_pred sum: 12
Y_train sum: 258
noun:0.5166960104141209
Baseline: 0.1092184368737475
diff: 0.0007346289103663528
feature: jcn
n->not_merged:2596, merged:367
shape of Y :(2963,)
shape of X :(2963, 11)
shape of Y :(2074,)
shape of X :(2074, 11)
y_pred sum: 10
Y_train sum: 258
noun:0.5091656915930929
Baseline: 0.1092184368737475
diff: 0.008264947731394412
feature: leskvalue
n->not_merged:2596, merged:367
shape of Y :(2963,)
shape of X :(2963, 11)
shape of Y :(2074,)
shape of X :(2074, 11)
y_pred sum: 8
Y_train sum: 258
noun:0.5013764748864087
Baseline: 0.1092184368737475
diff: 0.01605416443807861
feature: linvalue
n->not_m

v->not_merged:9876, merged:2751
n->not_merged:2596, merged:367
shape of Y :(12627,)
shape of X :(12627, 14)
shape of Y :(8838,)
shape of X :(8838, 14)
verb:0.4777483363645664
Baseline: 0.18340517241379312
shape of Y :(2963,)
shape of X :(2963, 12)
shape of Y :(2074,)
shape of X :(2074, 12)
noun:0.4673457159976034
Baseline: 0.1092184368737475


In [135]:
pru_col

['Pos',
 'Sense1',
 'Sense2',
 'Merge',
 'lch',
 'jcn',
 'leskvalue',
 'linvalue',
 'resvalue',
 'vecvalue',
 'wupvalue',
 'antonyms',
 'deriv',
 'lemmas',
 'wn_oed_feature']

In [143]:
test=np.array([1,0,0,1,1])
np.sum(test)

3

In [24]:
#verb features analysis
df_features_v = df_features[df_features['Pos']=='v'][['Pos', 'Sense1', 'Sense2', 'Merge', 
       'lch', 'hso', 'jcn','leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe','wn_oed_feature']]

v_columns=['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 'antonyms', 'deriv',
       'lemmas', 'verbgroup', 'verbframe', 'wn_oed_feature']
#replace labels with 0 or 1
for feature in v_columns[4:]:
    print('feature:',feature)
    df_features_v = df_features[df_features['Pos']=='v'][['Pos', 'Sense1', 'Sense2', 'Merge', 
       'lch', 'hso', 'jcn','leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas', 'verbgroup', 'verbframe','wn_oed_feature']]
    pru_col_v=['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 'antonyms', 'deriv',
       'lemmas', 'verbgroup', 'verbframe', 'wn_oed_feature']

    pru_col_v.remove(feature)
    df_features_v=df_features_v[pru_col_v]
    df_features_v=df_features_v.replace(['not-merged','merged'],[0,1])
    v_nm=len(df_features_v[df_features_v['Merge']==0])
    v_m=len(df_features_v[df_features_v['Merge']==1])
    F1_v_c=Fscore(df_pos=df_features_v, tune_flag='N', c=10,pos='verb')
    print('differ:',F1_v-F1_v_c)
    
    

feature: lch
shape of Y :(12627,)
shape of X :(12627, 13)
shape of Y :(8838,)
shape of X :(8838, 13)
y_pred sum: 144
Y_train sum: 1900
verb:0.535491982165117
Baseline: 0.18340517241379312
differ: -0.002752587978757437
feature: hso
shape of Y :(12627,)
shape of X :(12627, 13)
shape of Y :(8838,)
shape of X :(8838, 13)
y_pred sum: 129
Y_train sum: 1900
verb:0.5230085183512628
Baseline: 0.18340517241379312
differ: 0.009730875835096686
feature: jcn
shape of Y :(12627,)
shape of X :(12627, 13)
shape of Y :(8838,)
shape of X :(8838, 13)
y_pred sum: 147
Y_train sum: 1900
verb:0.5395621029292629
Baseline: 0.18340517241379312
differ: -0.006822708742903361
feature: leskvalue
shape of Y :(12627,)
shape of X :(12627, 13)
shape of Y :(8838,)
shape of X :(8838, 13)
y_pred sum: 144
Y_train sum: 1900
verb:0.5366489137163869
Baseline: 0.18340517241379312
differ: -0.003909519530027383
feature: linvalue
shape of Y :(12627,)
shape of X :(12627, 13)
shape of Y :(8838,)
shape of X :(8838, 13)
y_pred sum: 11

In [128]:
df_features_v.columns


Index(['Pos', 'Sense1', 'Sense2', 'Merge', 'lch', 'hso', 'jcn', 'leskvalue',
       'linvalue', 'resvalue', 'vecvalue', 'wupvalue', 'antonyms', 'deriv',
       'lemmas', 'verbgroup', 'verbframe', 'wn_oed_feature'],
      dtype='object')

## by adding SENSEVAL-2 data, F1 score decreasing
verb: 0.4819
noun: 0.4654 

In [ ]:
#solve always the same value
df_features_n = df_features[df_features['Pos']=='n'][['Pos', 'Sense1', 'Sense2', 'Merge', 
        'lch', 'hso', 'jcn', 'leskvalue', 'linvalue', 'resvalue', 'vecvalue', 'wupvalue',
       'antonyms', 'deriv', 'lemmas','wn_oed_feature']]

df_features_n=df_features_n.replace(['not-merged','merged'],[0,1])
df_pos= df_features_n
Y=np.array(df_pos['Merge'])
X=df_pos.drop(['Pos','Sense1','Sense2','Merge'],axis=1)
X=np.array(X)
#features_names=['1','2','3','4','5','6','7','8','9','10','11','12','13','14']
print("shape of Y :"+str(Y.shape))
print("shape of X :"+str(X.shape))
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
C=[1e4,1e5,1e6,1e7,1e8]
gammas=[1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
X_train,X_test,Y_train,Y_test = split_data(X,Y,0.3)
for c in C:
    
    #for g in gammas:
        #print('c=',c,'gamma=',g)
    clf1 = SVC(kernel='rbf',C=c)
    clf1.fit(X_train,Y_train)
    y_pr=clf1.predict(X_train)
    correct=0
    ones=0
    for i in range(len(Y_train)):
        if Y_train[i]==y_pr[i]:
            correct+=1
            if y_pr[i]==1:
                ones+=1
    print('c:{},correct:{},ones:{}'.format(c,correct,ones))

shape of Y :(2963,)
shape of X :(2963, 12)
c:10000.0,correct:1910,ones:94
c:100000.0,correct:1944,ones:133
c:1000000.0,correct:1970,ones:159
c:10000000.0,correct:1944,ones:167


In [178]:
1e2

100.0